In [ ]:
!pip install urllib3

!pip install tqdm

!pip install wheel --user
!pip install pipwin --user

!pipwin install numpy --user
!pipwin install pandas  --user
!pipwin install shapely --user
!pipwin install gdal  --user
!pipwin install fiona --user
!pipwin install pyproj --user
!pipwin install six --user
!pipwin install rtree --user
!pipwin install geopandas --user

!pip install xlrd

!pip install geopandas

In [1]:
import pandas as pd
import geopandas as gpd
import re 
from tqdm import tqdm

## Partie 1 : récupération des données 

Notre table de base est un fichier csv des établissements dans le domaine de la restauration à l'échelle de la France. Il contient un certain nombre d'informations, notamment l'adresse de l'établissement, son annéee de fondation, son nom, sa taille et enfin son numéro de siren (variable legalImmat).

In [2]:
df=pd.read_csv("C:/Users/GUILLOT Robin/Documents/Robin Ensae/Matières/Python/fichier_restaurants.csv",sep=';',header=8,error_bad_lines= False, warn_bad_lines=True)

In [3]:
df

,legalImmat,country,streetNumber,streetName,postalCode,city,isHeadquarters,categoryType,foundingYearSubs,legalName,...,legalIsNonprofit,employeesNb,categorySize,foundingYear,sourceLastUpdate,name,stateCode,financeLastDate,financeLastRevenue,financeLastProfit
0,7340516,FRANCE,NaN,NaN,4250.0,CLAMENSANE,1,Commerçant,1973.0,ROUX*MEGY/AIME/,...,NaN,0.0,PME,1973,2008-05-26T00:00:00,ROUX*MEGY AIME,4.0,NaN,NaN,NaN
1,46412375,FRANCE,21.0,RUE GEN DE GAULLE,20110.0,PROPRIANO,1,Commerçant,1988.0,SORBA*JACQUES DOMINIQUE/,...,NaN,0.0,PME,1964,2015-01-08T00:00:00,SORBA*JACQUES DOMINIQUE,20.0,NaN,NaN,NaN
2,66304932,FRANCE,NaN,COLLEGE DE KOUNGOU,97600.0,KOUNGOU,1,Commerçant,2005.0,NAFOUENTI*ASSANE/,...,NaN,NaN,PME,2005,2011-11-14T00:00:00,NAFOUENTI*ASSANE,97.0,NaN,NaN,NaN
3,76912112,FRANCE,NaN,LE COIN,73150.0,VAL D'ISERE,0,Autre entrepreneur individuel,2004.0,CHERRIER*JEAN-CLAUDE GEORGES/,...,NaN,1.0,PME,1968,2005-12-22T00:00:00,CHERRIER*JEAN-CLAUDE GEORGES,73.0,NaN,NaN,NaN
4,300443124,FRANCE,NaN,LIEU-DIT CAVALLO MORTO,20169.0,BONIFACIO,1,Commerçant,1973.0,COME*MICHEL HENRI/,...,NaN,1.0,PME,1973,2007-04-17T00:00:00,COME*MICHEL HENRI,20.0,NaN,NaN,NaN
5,301341517,FRANCE,1.0,ESPLANADE PARMENTIER,62600.0,BERCK,1,NaN,1974.0,SAS LA PLAGE,...,NaN,6.0,PME,1974,2016-01-05T00:00:00,SAS LA PLAGE,62.0,2015-03-31,916866.0,81676.0
6,301863668,FRANCE,NaN,BOIS DE COULANGE,57360.0,AMNEVILLE,1,NaN,1973.0,ASSOCIATION CENTRE LOISIRS D AMNEVILLE,...,1.0,50.0,PME,1900,2013-08-03T00:00:00,ASSOCIATION CENTRE LOISIRS D AMNEVILLE,57.0,NaN,NaN,NaN
7,302296629,FRANCE,146.0,RUE ROBESPIERRE,29200.0,BREST,1,Artisan-commerçant,1965.0,BIAN*APPRIOU/MARIE THERESE/,...,NaN,0.0,PME,1965,2013-11-25T00:00:00,BIAN*APPRIOU MARIE THERESE,29.0,NaN,NaN,NaN
8,303406540,FRANCE,9.0,AVENUE DE STALINGRAD,92700.0,COLOMBES,1,Commerçant,1992.0,TEBANE*ABDALLAH/,...,NaN,1.0,PME,1975,2013-08-19T00:00:00,TEBANE*ABDALLAH,92.0,NaN,NaN,NaN
9,303797823,FRANCE,NaN,LA MARE AUX PRETRES,71500.0,SORNAY,1,Commerçant,1995.0,METHY*LOISY/PAULETTE/,...,NaN,0.0,PME,1995,1995-06-09T00:00:00,METHY*LOISY PAULETTE,71.0,NaN,NaN,NaN


In [4]:
df=df.astype({'postalCode': str, 'legalImmat': str, 'city':str})
df=df.drop(columns=["country","foundingYearSubs"],axis=1)
df['postalCode']=df['postalCode'].apply(lambda s: s[:5] if len(s)>=7 else '0'+s[:4])
df['dep']=df['postalCode'].str[:2]

##### On va le compléter par diveses informations : 
    - des informations géographiques sur la commune de localisation du restaurant : population, revenu moyen par habitant, informations touristiques
    - des informations internes au restaurant : capital social, prix moyen pratiqué, lien TripAdvisor, notes et avis reçus

### Informations géographiques

On récupére sur le site de l'INSEE un fichier recensant les communes et leur population : 

In [8]:
population_communes=pd.read_csv("C:/Users/GUILLOT Robin/Documents/Robin Ensae/Matières/Python/Communes.csv",sep=';',error_bad_lines= False, warn_bad_lines=True)

In [9]:
population_communes

,DEPCOM,COM,PMUN,PCAP,PTOT
0,01001,L' Abergement-Clémenciat,776,18,794
1,01002,L' Abergement-de-Varey,248,1,249
2,01004,Ambérieu-en-Bugey,14035,393,14428
3,01005,Ambérieux-en-Dombes,1689,34,1723
4,01006,Ambléon,111,6,117
5,01007,Ambronay,2726,115,2841
6,01008,Ambutrix,752,15,767
7,01009,Andert-et-Condon,330,9,339
8,01010,Anglefort,1115,17,1132
9,01011,Apremont,376,11,387


In [10]:
population_communes=population_communes.rename(columns = {'DEPCOM': 'Code INSEE'})

Sur cette table, c'est le code INSEE qui est indiqué, (on a renommé juste au-dessus la variable DEPCOM en Code Insee) alors que sur notre table des restaurants, il s'agit du Code Postal. On ne peut donc pas directement faire la jointure ; il nous faut une table de correspondance entre code postal et Code Insee, qu'on récupère aussi sur le site de l'INSEE.

In [5]:
communes_insee=pd.read_csv("C:/Users/GUILLOT Robin/Documents/Robin Ensae/Matières/Python/correspondance-code-insee-code-postal.csv",sep=';')

In [6]:
communes_insee

,Code INSEE,Code Postal,Commune,Département,Région,Statut,Altitude Moyenne,Superficie,Population,geo_point_2d,geo_shape,ID Geofla,Code Commune,Code Canton,Code Arrondissement,Code Département,Code Région
0,59276,59287,GUESNAIN,NORD,NORD-PAS-DE-CALAIS,Commune simple,27.0,405.0,4.6,"50.3483461671,3.14816711364","{""type"": ""Polygon"", ""coordinates"": [[[3.135757...",13979,276,24,3,59,31
1,88128,88210,DENIPAIRE,VOSGES,LORRAINE,Commune simple,425.0,702.0,0.3,"48.3398168543,6.96189290978","{""type"": ""Polygon"", ""coordinates"": [[[6.971447...",30622,128,26,3,88,41
2,57538,57170,PETTONCOURT,MOSELLE,LORRAINE,Commune simple,224.0,493.0,0.3,"48.7881263595,6.41106883774","{""type"": ""Polygon"", ""coordinates"": [[[6.394874...",29834,538,6,2,57,41
3,54459,54630,RICHARDMENIL,MEURTHE-ET-MOSELLE,LORRAINE,Commune simple,257.0,707.0,2.5,"48.5958649273,6.17614506031","{""type"": ""Polygon"", ""coordinates"": [[[6.164140...",15794,459,25,3,54,41
4,35022,35190,BECHEREL,ILLE-ET-VILAINE,BRETAGNE,Chef-lieu canton,148.0,55.0,0.7,"48.2965087434,-1.9428236566","{""type"": ""Polygon"", ""coordinates"": [[[-1.94802...",12793,22,4,3,35,53
5,21459,21310,NOIRON-SUR-BEZE,COTE-D'OR,BOURGOGNE,Commune simple,216.0,1181.0,0.2,"47.4394112405,5.29872318112","{""type"": ""Polygon"", ""coordinates"": [[[5.316173...",7181,459,20,2,21,26
6,33108,33350,CASTILLON-LA-BATAILLE,GIRONDE,AQUITAINE,Chef-lieu canton,39.0,569.0,3.2,"44.8619054717,-0.0287795124609","{""type"": ""Polygon"", ""coordinates"": [[[-0.05004...",18482,108,21,5,33,72
7,25088,25640,BRECONCHAUX,DOUBS,FRANCHE-COMTE,Commune simple,362.0,331.0,0.1,"47.3418237888,6.27061644005","{""type"": ""Polygon"", ""coordinates"": [[[6.266896...",29517,88,24,1,25,43
8,47120,47180,JUSIX,LOT-ET-GARONNE,AQUITAINE,Commune simple,16.0,748.0,0.1,"44.5392859765,0.0488579062154","{""type"": ""Polygon"", ""coordinates"": [[[0.051958...",21141,120,21,2,47,72
9,39056,39140,BLETTERANS,JURA,FRANCHE-COMTE,Chef-lieu canton,198.0,780.0,1.4,"46.7316211339,5.42781411933","{""type"": ""Polygon"", ""coordinates"": [[[5.406641...",10259,56,4,2,39,43


In [7]:
corresp=communes_insee.drop(columns=['geo_point_2d','geo_shape','ID Geofla','Code Commune','Code Canton','Code Arrondissement','Code Région','Code Département','Statut'],axis=1)

On peut faire la jointure entre nos deux tables population et correspondance code INSEE, code Postal (clé commune = Code INSEE) :

In [11]:
pop_superf=pd.merge(corresp,population_communes)
pop_superf=pop_superf.drop(columns=['COM','PMUN','PCAP','Population'],axis=1)

Notre nouvelle table pop_superf contient désormais de nombreuses informations géographiques sur les communes : altitude, superficie, et surtout population.

In [12]:
pop_superf

,Code INSEE,Code Postal,Commune,Département,Région,Altitude Moyenne,Superficie,PTOT
0,59276,59287,GUESNAIN,NORD,NORD-PAS-DE-CALAIS,27.0,405.0,4713
1,88128,88210,DENIPAIRE,VOSGES,LORRAINE,425.0,702.0,252
2,57538,57170,PETTONCOURT,MOSELLE,LORRAINE,224.0,493.0,293
3,54459,54630,RICHARDMENIL,MEURTHE-ET-MOSELLE,LORRAINE,257.0,707.0,2380
4,35022,35190,BECHEREL,ILLE-ET-VILAINE,BRETAGNE,148.0,55.0,680
5,21459,21310,NOIRON-SUR-BEZE,COTE-D'OR,BOURGOGNE,216.0,1181.0,243
6,33108,33350,CASTILLON-LA-BATAILLE,GIRONDE,AQUITAINE,39.0,569.0,3202
7,25088,25640,BRECONCHAUX,DOUBS,FRANCHE-COMTE,362.0,331.0,98
8,47120,47180,JUSIX,LOT-ET-GARONNE,AQUITAINE,16.0,748.0,119
9,39056,39140,BLETTERANS,JURA,FRANCHE-COMTE,198.0,780.0,1465


Il nous faut maintenant des informations sur le niveau de vie des communes : on récupère un fichier csv de l'INSEE qui indique pour chaque commune son "niveau de vie" = revenu moyen par habitant.

In [14]:
revenus_com=pd.read_excel("C:/Users/GUILLOT Robin/Documents/Robin Ensae/Matières/Python/Niveau_de_vie_2013_a_la_commune-Global_Map_Solution.xlsx")

In [15]:
revenus_com

,Code Commune,Nom Commune,Niveau de vie Commune,Niveau de vie Département
0,05047,Éourres,10021.250000,19202.516480
1,26142,Glandage,10215.000000,19300.504181
2,11317,Rodome,10908.500000,17599.733068
3,11384,Soulatgé,11485.166667,17599.733068
4,30153,Malons-et-Elze,11680.000000,18282.543539
5,13203,Marseille 3e Arrondissement,11760.800000,20006.785169
6,26055,Boulc,11903.214286,19300.504181
7,09118,Esplas-de-Sérou,12074.000000,18463.189424
8,11346,Saint-Jean-de-Paracol,12173.333333,17599.733068
9,09231,Le Port,12193.000000,18463.189424


In [16]:
revenus_com=revenus_com.rename(columns={'Code Commune':'Code INSEE'})

On joint cette table à notre dataframe précédent pop_superf (clé commune = Code INSEE toujours). On obtient un dataframe plus complet, nommé intemed.

In [17]:
intemed=pd.merge(pop_superf,revenus_com,how='left')
intemed=intemed.drop(columns=['Nom Commune'],axis=1)

In [18]:
intemed

,Code INSEE,Code Postal,Commune,Département,Région,Altitude Moyenne,Superficie,PTOT,Niveau de vie Commune,Niveau de vie Département
0,59276,59287,GUESNAIN,NORD,NORD-PAS-DE-CALAIS,27.0,405.0,4713,16277.333333,18710.357368
1,88128,88210,DENIPAIRE,VOSGES,LORRAINE,425.0,702.0,252,20700.000000,18846.767927
2,57538,57170,PETTONCOURT,MOSELLE,LORRAINE,224.0,493.0,293,20672.777778,20213.474303
3,54459,54630,RICHARDMENIL,MEURTHE-ET-MOSELLE,LORRAINE,257.0,707.0,2380,25902.222222,20148.048011
4,35022,35190,BECHEREL,ILLE-ET-VILAINE,BRETAGNE,148.0,55.0,680,18340.000000,20791.099979
5,21459,21310,NOIRON-SUR-BEZE,COTE-D'OR,BOURGOGNE,216.0,1181.0,243,22481.500000,20875.574546
6,33108,33350,CASTILLON-LA-BATAILLE,GIRONDE,AQUITAINE,39.0,569.0,3202,14422.000000,20905.253619
7,25088,25640,BRECONCHAUX,DOUBS,FRANCHE-COMTE,362.0,331.0,98,NaN,20760.808968
8,47120,47180,JUSIX,LOT-ET-GARONNE,AQUITAINE,16.0,748.0,119,17638.571429,18468.493000
9,39056,39140,BLETTERANS,JURA,FRANCHE-COMTE,198.0,780.0,1465,18147.500000,20029.655806


Enfin, on souhaite récupérer des informations touristiques sur nos communes, car on pense bien qu'un restaurant situé dans des communes plus touristiques que d'autres est à même de pratiquer des prix plus élevés.

On récupère un fichier excel, toujours sur le site de l'INSEE

In [19]:
tourist_communes=pd.ExcelFile("C:/Users/GUILLOT Robin/Documents/Robin Ensae/Matières/Python/base-cc-tourisme-2020.xlsx")

Ce fichier est assez dense, contient 4 feuilles excel différentes comme on peut le voir juste en-dessous. Seul nous intéresse les deux premières feuille excel : 'COM', qui contient les données recherchées pour les communes, et 'ARM', qui contient les mêmes données mais pour les arrondissements des 3 plus grandes villes (Lyon/Paris/Marseille).

In [20]:
print(tourist_communes.sheet_names)

['COM', 'ARM', 'Variables', 'Documentation']


In [21]:
communes_touristiques=pd.read_excel(tourist_communes,'COM',header=5)
arrondissements_touristiques=pd.read_excel(tourist_communes,'ARM',header=5)

In [22]:
arrondissements_touristiques=arrondissements_touristiques.astype({'CODGEO': str})

In [23]:
tourisme=pd.concat([communes_touristiques,arrondissements_touristiques])

In [24]:
keep_col=['CODGEO','LIBGEO','HT20','HT120','HT220','HT320','HT420','HT520','CPG20','RT20']
tourisme = tourisme[[c for c in tourisme.columns if c in keep_col]]
tourisme=tourisme.rename(columns={'CODGEO':'Code INSEE'})

##### On ne garde que les principales variables touristiques qui vont nous intéresser, à savoir : 
    - HT20 : nombre d'hôtels dans la commune en question en 2020
    - HT120 : nombre d'hôtels 1 étoile dans la commune en 2020
    - HT220 : nombre d'hôtels 2 étoiles
        ...
        
    - CPG20 : nombre de campings dans la commune en 2020
    - RT20 : nombre de résidence de tourisme et hébergements assimilés dans la commune en 2020

In [25]:
tourisme

,Code INSEE,LIBGEO,HT20,HT120,HT220,HT320,HT420,HT520,CPG20,RT20
0,01001,L'Abergement-Clémenciat,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,01002,L'Abergement-de-Varey,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,01004,Ambérieu-en-Bugey,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,01005,Ambérieux-en-Dombes,2.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,01006,Ambléon,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,01007,Ambronay,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,01008,Ambutrix,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,01009,Andert-et-Condon,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,01010,Anglefort,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,01011,Apremont,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


On joint cette table à la précédente, toujours sur la clé commune Code INSEE

In [26]:
donnees_externes=pd.merge(intemed,tourisme)

On a désormais toute une table complète de données géographiques contenant diverses informations sur les communes

In [27]:
donnees_externes

,Code INSEE,Code Postal,Commune,Département,Région,Altitude Moyenne,Superficie,PTOT,Niveau de vie Commune,Niveau de vie Département,LIBGEO,HT20,HT120,HT220,HT320,HT420,HT520,CPG20,RT20
0,59276,59287,GUESNAIN,NORD,NORD-PAS-DE-CALAIS,27.0,405.0,4713,16277.333333,18710.357368,Guesnain,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,88128,88210,DENIPAIRE,VOSGES,LORRAINE,425.0,702.0,252,20700.000000,18846.767927,Denipaire,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,57538,57170,PETTONCOURT,MOSELLE,LORRAINE,224.0,493.0,293,20672.777778,20213.474303,Pettoncourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,54459,54630,RICHARDMENIL,MEURTHE-ET-MOSELLE,LORRAINE,257.0,707.0,2380,25902.222222,20148.048011,Richardménil,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,35022,35190,BECHEREL,ILLE-ET-VILAINE,BRETAGNE,148.0,55.0,680,18340.000000,20791.099979,Bécherel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,21459,21310,NOIRON-SUR-BEZE,COTE-D'OR,BOURGOGNE,216.0,1181.0,243,22481.500000,20875.574546,Noiron-sur-Bèze,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,33108,33350,CASTILLON-LA-BATAILLE,GIRONDE,AQUITAINE,39.0,569.0,3202,14422.000000,20905.253619,Castillon-la-Bataille,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7,25088,25640,BRECONCHAUX,DOUBS,FRANCHE-COMTE,362.0,331.0,98,NaN,20760.808968,Breconchaux,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,47120,47180,JUSIX,LOT-ET-GARONNE,AQUITAINE,16.0,748.0,119,17638.571429,18468.493000,Jusix,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,39056,39140,BLETTERANS,JURA,FRANCHE-COMTE,198.0,780.0,1465,18147.500000,20029.655806,Bletterans,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
donnees_externes=donnees_externes.drop(columns=['LIBGEO'],axis=1)

Maintenant, on joint les donnees_externes à df :
la colonne commune est le code postal

In [29]:
df=df.rename(columns={'postalCode': 'Code Postal'})

In [30]:
df

,legalImmat,streetNumber,streetName,Code Postal,city,isHeadquarters,categoryType,legalName,contactName,contactFirstname,...,employeesNb,categorySize,foundingYear,sourceLastUpdate,name,stateCode,financeLastDate,financeLastRevenue,financeLastProfit,dep
0,7340516,NaN,NaN,04250,CLAMENSANE,1,Commerçant,ROUX*MEGY/AIME/,ROUX,AIME,...,0.0,PME,1973,2008-05-26T00:00:00,ROUX*MEGY AIME,4.0,NaN,NaN,NaN,04
1,46412375,21.0,RUE GEN DE GAULLE,20110,PROPRIANO,1,Commerçant,SORBA*JACQUES DOMINIQUE/,SORBA,JACQUES,...,0.0,PME,1964,2015-01-08T00:00:00,SORBA*JACQUES DOMINIQUE,20.0,NaN,NaN,NaN,20
2,66304932,NaN,COLLEGE DE KOUNGOU,97600,KOUNGOU,1,Commerçant,NAFOUENTI*ASSANE/,NAFOUENTI,ASSANE,...,NaN,PME,2005,2011-11-14T00:00:00,NAFOUENTI*ASSANE,97.0,NaN,NaN,NaN,97
3,76912112,NaN,LE COIN,73150,VAL D'ISERE,0,Autre entrepreneur individuel,CHERRIER*JEAN-CLAUDE GEORGES/,CHERRIER,JEAN-CLAUDE,...,1.0,PME,1968,2005-12-22T00:00:00,CHERRIER*JEAN-CLAUDE GEORGES,73.0,NaN,NaN,NaN,73
4,300443124,NaN,LIEU-DIT CAVALLO MORTO,20169,BONIFACIO,1,Commerçant,COME*MICHEL HENRI/,COME,MICHEL,...,1.0,PME,1973,2007-04-17T00:00:00,COME*MICHEL HENRI,20.0,NaN,NaN,NaN,20
5,301341517,1.0,ESPLANADE PARMENTIER,62600,BERCK,1,NaN,SAS LA PLAGE,NaN,NaN,...,6.0,PME,1974,2016-01-05T00:00:00,SAS LA PLAGE,62.0,2015-03-31,916866.0,81676.0,62
6,301863668,NaN,BOIS DE COULANGE,57360,AMNEVILLE,1,NaN,ASSOCIATION CENTRE LOISIRS D AMNEVILLE,NaN,NaN,...,50.0,PME,1900,2013-08-03T00:00:00,ASSOCIATION CENTRE LOISIRS D AMNEVILLE,57.0,NaN,NaN,NaN,57
7,302296629,146.0,RUE ROBESPIERRE,29200,BREST,1,Artisan-commerçant,BIAN*APPRIOU/MARIE THERESE/,BIAN,MARIE,...,0.0,PME,1965,2013-11-25T00:00:00,BIAN*APPRIOU MARIE THERESE,29.0,NaN,NaN,NaN,29
8,303406540,9.0,AVENUE DE STALINGRAD,92700,COLOMBES,1,Commerçant,TEBANE*ABDALLAH/,TEBANE,ABDALLAH,...,1.0,PME,1975,2013-08-19T00:00:00,TEBANE*ABDALLAH,92.0,NaN,NaN,NaN,92
9,303797823,NaN,LA MARE AUX PRETRES,71500,SORNAY,1,Commerçant,METHY*LOISY/PAULETTE/,METHY,PAULETTE,...,0.0,PME,1995,1995-06-09T00:00:00,METHY*LOISY PAULETTE,71.0,NaN,NaN,NaN,71


In [32]:
etablissements=pd.merge(df,donnees_externes,how='left')

In [33]:
etablissements

,legalImmat,streetNumber,streetName,Code Postal,city,isHeadquarters,categoryType,legalName,contactName,contactFirstname,...,Niveau de vie Commune,Niveau de vie Département,HT20,HT120,HT220,HT320,HT420,HT520,CPG20,RT20
0,7340516,NaN,NaN,04250,CLAMENSANE,1,Commerçant,ROUX*MEGY/AIME/,ROUX,AIME,...,NaN,19008.420673,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7340516,NaN,NaN,04250,CLAMENSANE,1,Commerçant,ROUX*MEGY/AIME/,ROUX,AIME,...,NaN,19008.420673,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,7340516,NaN,NaN,04250,CLAMENSANE,1,Commerçant,ROUX*MEGY/AIME/,ROUX,AIME,...,16323.333333,19008.420673,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,7340516,NaN,NaN,04250,CLAMENSANE,1,Commerçant,ROUX*MEGY/AIME/,ROUX,AIME,...,18580.000000,19008.420673,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,7340516,NaN,NaN,04250,CLAMENSANE,1,Commerçant,ROUX*MEGY/AIME/,ROUX,AIME,...,16936.666667,19008.420673,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,7340516,NaN,NaN,04250,CLAMENSANE,1,Commerçant,ROUX*MEGY/AIME/,ROUX,AIME,...,18083.452381,19008.420673,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
6,7340516,NaN,NaN,04250,CLAMENSANE,1,Commerçant,ROUX*MEGY/AIME/,ROUX,AIME,...,NaN,19008.420673,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,7340516,NaN,NaN,04250,CLAMENSANE,1,Commerçant,ROUX*MEGY/AIME/,ROUX,AIME,...,NaN,19008.420673,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,7340516,NaN,NaN,04250,CLAMENSANE,1,Commerçant,ROUX*MEGY/AIME/,ROUX,AIME,...,NaN,19008.420673,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,7340516,NaN,NaN,04250,CLAMENSANE,1,Commerçant,ROUX*MEGY/AIME/,ROUX,AIME,...,NaN,19008.420673,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Probleme ! Car un code postal correspond à plusieurs communes, alors que le Code Insee correspond à une seule commune.
Par conséquent, on se trouve dans une table avec des numéros de siren répétés, car le code postal du restaurant correspondant est répété autant de fois qu'il existe de numéros insee ayant ce code postal


Illustration avec l'entreprise de siren ' 7340516 ' :


In [34]:
etablissements[etablissements['legalImmat']=='7340516']


,legalImmat,streetNumber,streetName,Code Postal,city,isHeadquarters,categoryType,legalName,contactName,contactFirstname,...,Niveau de vie Commune,Niveau de vie Département,HT20,HT120,HT220,HT320,HT420,HT520,CPG20,RT20
0,7340516,NaN,NaN,04250,CLAMENSANE,1,Commerçant,ROUX*MEGY/AIME/,ROUX,AIME,...,NaN,19008.420673,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7340516,NaN,NaN,04250,CLAMENSANE,1,Commerçant,ROUX*MEGY/AIME/,ROUX,AIME,...,NaN,19008.420673,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,7340516,NaN,NaN,04250,CLAMENSANE,1,Commerçant,ROUX*MEGY/AIME/,ROUX,AIME,...,16323.333333,19008.420673,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,7340516,NaN,NaN,04250,CLAMENSANE,1,Commerçant,ROUX*MEGY/AIME/,ROUX,AIME,...,18580.000000,19008.420673,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,7340516,NaN,NaN,04250,CLAMENSANE,1,Commerçant,ROUX*MEGY/AIME/,ROUX,AIME,...,16936.666667,19008.420673,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,7340516,NaN,NaN,04250,CLAMENSANE,1,Commerçant,ROUX*MEGY/AIME/,ROUX,AIME,...,18083.452381,19008.420673,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
6,7340516,NaN,NaN,04250,CLAMENSANE,1,Commerçant,ROUX*MEGY/AIME/,ROUX,AIME,...,NaN,19008.420673,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,7340516,NaN,NaN,04250,CLAMENSANE,1,Commerçant,ROUX*MEGY/AIME/,ROUX,AIME,...,NaN,19008.420673,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,7340516,NaN,NaN,04250,CLAMENSANE,1,Commerçant,ROUX*MEGY/AIME/,ROUX,AIME,...,NaN,19008.420673,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,7340516,NaN,NaN,04250,CLAMENSANE,1,Commerçant,ROUX*MEGY/AIME/,ROUX,AIME,...,NaN,19008.420673,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Ici, il y 12 lignes pour le même établissement, car le code postal "04250" correspond à 12 communes différentes :

In [35]:
etablissements[etablissements['legalImmat']=='7340516']['Commune']

0              LE CAIRE
1                GIGORS
2                BAYONS
3           BELLAFFAIRE
4              TURRIERS
5            CLAMENSANE
6           CHATEAUFORT
7             VALAVOIRE
8                NIBLES
9       FAUCON-DU-CAIRE
10    LA MOTTE-DU-CAIRE
11                MELVE
Name: Commune, dtype: object

 Notre établissement est situé dans la commune de "CLAMENSANE" ; il faut donc choisir la bonne ligne parmi les douzes, c'est-à-dire celle qui correspond à "CLAMENSANE"
 
 Le plus simple est de refaire la jointure entre df et donnees_exterieures, en rajoutant la condition "même nom de commune" (le nom de la commune étant présent dans les deux tables qu'on veut joindre). Cependant, et puisque certains noms de communes posent problème avec des tirets dans une table et non présents dans l'autre, on va dans chaque table créer la variabe id_nom qui transforme le nom de la commune en une suite de caractères collés (c'est-à-dire on supprime les espaces, apostrophes ou tirets).

Si on tombe sur un arrondissement de Paris, Lyon ou Marseille, la variable prend le code postal comme valeur (en fait ces trois villes ne posent pas problème pour la jointure car il y a un unique code postal pour chaque arrondissement de ces villes). cependant, on ne leur applique pas le même traitement que les autres (à savoir supprimer espaces, tirets, apostrophes), car dans la table df, leur nom est du type "Paris5", alors que dans la table donnees_externes, leur nom est du type "Paris5EArrondissement", ce qui nécessiterait de faire encore d'autres opérations inutiles pour avoir les mêmes noms 
 
 
 

In [36]:
# liste des codes Postaux des arrondissements de nos trois villes

liste_code=['75001','75002','75003','75004','75005','75006','75007','75008','75009','75010',
           '75011','75012','75013','75014','75015','75016','75017','75018','75019','75020',
           '69001','69002','69003','69004','69005','69006','69007','69008','69009','13001',
          '13002','13003','13004','13005','13006','13007','13008','13009','13010','13011',
          '13012','13013','13014','13015','13016']

donnees_externes['id_nom']=donnees_externes.apply(lambda s: re.sub('[-\'\s]','',s['Commune']) if s['Code Postal'] not in liste_code else s['Code Postal'],axis=1)

df['id_nom']=df.apply(lambda s: re.sub('[-\'\s]','',s['city']) if s['Code Postal'] not in liste_code else s['Code Postal'],axis=1)


On peut désormais faire notre jointure de df et donnees_externes : celle-ci se fera sur les deux colonnes communes code_postal et id_nom


In [37]:
etablissements=pd.merge(df,donnees_externes,how='left')

In [39]:
etablissements

,legalImmat,streetNumber,streetName,Code Postal,city,isHeadquarters,categoryType,legalName,contactName,contactFirstname,...,Niveau de vie Commune,Niveau de vie Département,HT20,HT120,HT220,HT320,HT420,HT520,CPG20,RT20
0,7340516,NaN,NaN,04250,CLAMENSANE,1,Commerçant,ROUX*MEGY/AIME/,ROUX,AIME,...,18083.452381,19008.420673,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
1,46412375,21.0,RUE GEN DE GAULLE,20110,PROPRIANO,1,Commerçant,SORBA*JACQUES DOMINIQUE/,SORBA,JACQUES,...,16608.888889,19525.216810,11.0,0.0,1.0,6.0,1.0,1.0,1.0,6.0
2,66304932,NaN,COLLEGE DE KOUNGOU,97600,KOUNGOU,1,Commerçant,NAFOUENTI*ASSANE/,NAFOUENTI,ASSANE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,76912112,NaN,LE COIN,73150,VAL D'ISERE,0,Autre entrepreneur individuel,CHERRIER*JEAN-CLAUDE GEORGES/,CHERRIER,JEAN-CLAUDE,...,22323.600000,21166.752819,32.0,1.0,4.0,10.0,5.0,7.0,1.0,9.0
4,300443124,NaN,LIEU-DIT CAVALLO MORTO,20169,BONIFACIO,1,Commerçant,COME*MICHEL HENRI/,COME,MICHEL,...,18240.400000,19525.216810,23.0,1.0,1.0,7.0,5.0,1.0,9.0,12.0
5,301341517,1.0,ESPLANADE PARMENTIER,62600,BERCK,1,NaN,SAS LA PLAGE,NaN,NaN,...,17398.039216,17738.487318,9.0,0.0,5.0,1.0,0.0,0.0,10.0,1.0
6,301863668,NaN,BOIS DE COULANGE,57360,AMNEVILLE,1,NaN,ASSOCIATION CENTRE LOISIRS D AMNEVILLE,NaN,NaN,...,19573.333333,20213.474303,6.0,0.0,1.0,4.0,1.0,0.0,0.0,4.0
7,302296629,146.0,RUE ROBESPIERRE,29200,BREST,1,Artisan-commerçant,BIAN*APPRIOU/MARIE THERESE/,BIAN,MARIE,...,19050.869565,20247.768730,29.0,1.0,9.0,11.0,6.0,0.0,1.0,3.0
8,303406540,9.0,AVENUE DE STALINGRAD,92700,COLOMBES,1,Commerçant,TEBANE*ABDALLAH/,TEBANE,ABDALLAH,...,20564.444444,26918.903409,3.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0
9,303797823,NaN,LA MARE AUX PRETRES,71500,SORNAY,1,Commerçant,METHY*LOISY/PAULETTE/,METHY,PAULETTE,...,19124.375000,19250.591565,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
list(etablissements)

['legalImmat',
 'streetNumber',
 'streetName',
 'Code Postal',
 'city',
 'isHeadquarters',
 'categoryType',
 'legalName',
 'contactName',
 'contactFirstname',
 'legalStructure',
 'categoryCode',
 'categoryLabel',
 'legalIsNonprofit',
 'employeesNb',
 'categorySize',
 'foundingYear',
 'sourceLastUpdate',
 'name',
 'stateCode',
 'financeLastDate',
 'financeLastRevenue',
 'financeLastProfit',
 'dep',
 'id_nom',
 'Code INSEE',
 'Commune',
 'Département',
 'Région',
 'Altitude Moyenne',
 'Superficie',
 'PTOT',
 'Niveau de vie Commune',
 'Niveau de vie Département',
 'HT20',
 'HT120',
 'HT220',
 'HT320',
 'HT420',
 'HT520',
 'CPG20',
 'RT20']

In [41]:
etablissements=etablissements.drop(columns=['Commune','id_nom','categoryCode','categoryLabel','legalIsNonprofit','stateCode','financeLastDate','financeLastRevenue','financeLastProfit'],axis=1)

In [42]:
etablissements

,legalImmat,streetNumber,streetName,Code Postal,city,isHeadquarters,categoryType,legalName,contactName,contactFirstname,...,Niveau de vie Commune,Niveau de vie Département,HT20,HT120,HT220,HT320,HT420,HT520,CPG20,RT20
0,7340516,NaN,NaN,04250,CLAMENSANE,1,Commerçant,ROUX*MEGY/AIME/,ROUX,AIME,...,18083.452381,19008.420673,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
1,46412375,21.0,RUE GEN DE GAULLE,20110,PROPRIANO,1,Commerçant,SORBA*JACQUES DOMINIQUE/,SORBA,JACQUES,...,16608.888889,19525.216810,11.0,0.0,1.0,6.0,1.0,1.0,1.0,6.0
2,66304932,NaN,COLLEGE DE KOUNGOU,97600,KOUNGOU,1,Commerçant,NAFOUENTI*ASSANE/,NAFOUENTI,ASSANE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,76912112,NaN,LE COIN,73150,VAL D'ISERE,0,Autre entrepreneur individuel,CHERRIER*JEAN-CLAUDE GEORGES/,CHERRIER,JEAN-CLAUDE,...,22323.600000,21166.752819,32.0,1.0,4.0,10.0,5.0,7.0,1.0,9.0
4,300443124,NaN,LIEU-DIT CAVALLO MORTO,20169,BONIFACIO,1,Commerçant,COME*MICHEL HENRI/,COME,MICHEL,...,18240.400000,19525.216810,23.0,1.0,1.0,7.0,5.0,1.0,9.0,12.0
5,301341517,1.0,ESPLANADE PARMENTIER,62600,BERCK,1,NaN,SAS LA PLAGE,NaN,NaN,...,17398.039216,17738.487318,9.0,0.0,5.0,1.0,0.0,0.0,10.0,1.0
6,301863668,NaN,BOIS DE COULANGE,57360,AMNEVILLE,1,NaN,ASSOCIATION CENTRE LOISIRS D AMNEVILLE,NaN,NaN,...,19573.333333,20213.474303,6.0,0.0,1.0,4.0,1.0,0.0,0.0,4.0
7,302296629,146.0,RUE ROBESPIERRE,29200,BREST,1,Artisan-commerçant,BIAN*APPRIOU/MARIE THERESE/,BIAN,MARIE,...,19050.869565,20247.768730,29.0,1.0,9.0,11.0,6.0,0.0,1.0,3.0
8,303406540,9.0,AVENUE DE STALINGRAD,92700,COLOMBES,1,Commerçant,TEBANE*ABDALLAH/,TEBANE,ABDALLAH,...,20564.444444,26918.903409,3.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0
9,303797823,NaN,LA MARE AUX PRETRES,71500,SORNAY,1,Commerçant,METHY*LOISY/PAULETTE/,METHY,PAULETTE,...,19124.375000,19250.591565,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
# On enregistre la table dans un fichier csv

etablissements.to_csv('C:/Users/GUILLOT Robin/Documents/Robin Ensae/Matières/Python/etablissements.csv', sep = ';',index=False)

In [11]:
etablissements=pd.read_csv('C:/Users/GUILLOT Robin/Documents/Robin Ensae/Matières/Python/etablissements.csv', sep = ';')

Nous avons donc la table établissements qui contient déjà pas mal d'informations sur les établissements, leur type, localisation...

Pour tout la suite de ce projet, nous allons nous concentrer sur les restaurants parisiens : 

In [58]:
parisiens=etablissements[etablissements['dep']=='75']
parisiens['arr']=parisiens['Code Postal'].apply(lambda s: s[3:] if s[3]!='0' else s[4:])

C:\Users\GUILLOT Robin\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [59]:
parisiens

,legalImmat,streetNumber,streetName,Code Postal,city,isHeadquarters,categoryType,legalName,contactName,contactFirstname,...,Niveau de vie Département,HT20,HT120,HT220,HT320,HT420,HT520,CPG20,RT20,arr
10,304305519,5.0,AVENUE DU MAINE,75015,PARIS 15,1,NaN,LES CEDRES DU LIBAN,NaN,NaN,...,27171.253619,98.0,1.0,9.0,60.0,23.0,0.0,0.0,7.0,15
17,306332362,25.0,RUE DE LA HUCHETTE,75005,PARIS 5,1,NaN,LE SOUVLAKI ATHENIEN,NaN,NaN,...,27171.253619,84.0,1.0,14.0,35.0,24.0,0.0,0.0,3.0,5
19,307711895,15.0,PLACE DU TERTRE,75018,PARIS 18,1,NaN,LA CREMAILLERE DE MONTMARTRE,NaN,NaN,...,27171.253619,67.0,2.0,10.0,23.0,12.0,0.0,0.0,5.0,18
37,315946814,3.0,RUE DE L'ANCIENNE COMEDIE,75006,PARIS 6,1,NaN,F A S E P,NaN,NaN,...,27171.253619,105.0,1.0,10.0,40.0,33.0,7.0,0.0,4.0,6
51,318906591,11.0,RUE DE L ARRIVEE,75015,PARIS 15,0,NaN,LA BRIOCHE DOREE,NaN,NaN,...,27171.253619,98.0,1.0,9.0,60.0,23.0,0.0,0.0,7.0,15
57,319947537,14.0,RUE MABILLON,75006,PARIS 6,1,NaN,BRASSERIE DE LA GRILLE SAINT-GERMAIN,NaN,NaN,...,27171.253619,105.0,1.0,10.0,40.0,33.0,7.0,0.0,4.0,6
58,320228919,26.0,RUE PONCELET,75017,PARIS 17,1,NaN,AFO,NaN,NaN,...,27171.253619,128.0,2.0,9.0,61.0,33.0,1.0,0.0,3.0,17
85,325621605,14.0,BOULEVARD DE CLICHY,75018,PARIS 18,1,Commerçant,DRAGONAS*DENIS/,DRAGONAS,DENIS,...,27171.253619,67.0,2.0,10.0,23.0,12.0,0.0,0.0,5.0,18
97,327848875,7.0,RUE SAINTE APOLLINE,75003,PARIS 3,1,Commerçant,SALLES*MICHELE/,SALLES,MICHELE,...,27171.253619,31.0,0.0,4.0,9.0,9.0,2.0,0.0,0.0,3
121,331307975,110.0,RUE DE COURCELLES,75017,PARIS 17,1,NaN,COURCELLES-MEDERIC,NaN,NaN,...,27171.253619,128.0,2.0,9.0,61.0,33.0,1.0,0.0,3.0,17


On va à cette table des établissements parisiens rajouter quelques informations. D'abord, on a pu récupérer sur le site de l'INSEE la localisation de tous les restaurants "émeraudes" de Paris, qui s'adressent aux personnes âgées essentiellement mais aussi aux personnes plus modestes, car ces restaurants pratiquent des prix selon les tranches de revenus
Il y en a en tout 43, voici la table (il s'agit là d'un fichier gson):

In [15]:
dl=gpd.read_file("C:/Users/GUILLOT Robin/Documents/Robin Ensae/Matières/Python/restaurants-casvp.geojson")

In [16]:
dl

,ville,code,adresse,nom_restaurant,type,geometry
0,PARIS,75002,"8, RUE SAINT-SAUVEUR",SAINT-SAUVEUR,E,POINT (2.34948 48.86593)
1,PARIS,75002,"28, RUE GRAMONT",GRAMONT,E,POINT (2.33703 48.87096)
2,PARIS,75013,"166, RUE DE TOLBIAC",TOLBIAC,E,POINT (2.35256 48.82603)
3,PARIS,75017,"51, RUE DES EPINETTES",EPINETTES,S,POINT (2.32334 48.89642)
4,PARIS,75010,"14, AVENUE DE VERDUN",VERDUN,E,POINT (2.36194 48.87627)
5,PARIS,75004,"4, RUE DE L'AVE MARIA",AVE MARIA,E,POINT (2.36053 48.85275)
6,PARIS,75008,"11, RUE MALLEVILLE",MARCHE DE L'EUROPE,S,POINT (2.31345 48.87728)
7,PARIS,75015,"18, RUE BARGUE",LA QUINTINIE,E,POINT (2.30718 48.83930)
8,PARIS,75003,"2, RUE AU MAIRE",AU MAIRE,E,POINT (2.35771 48.86445)
9,PARIS,75018,"88, RUE JOSEPH DE MAISTRE",JOSEPH DE MAISTRE,E,POINT (2.33046 48.89289)


In [17]:
def streetNumber(s):
    s=str(s)
    StNb=''
    for i in s:
        if i!=',':
            StNb+=i
        else:
            return StNb
        
def streetName(s):
    s=str(s)
    i=0
    while i<len(s):
        if s[i]==',':
            return s[i+1:]
        else:
            i+=1
        
            
print(streetName('8,rue Saint-Sauveur'))
print(streetNumber('8,rue Saint-sauveur'))    

rue Saint-Sauveur
8


In [18]:
dl['streetNumber']=dl['adresse'].apply(lambda s: streetNumber(s))
dl['streetName']=dl['adresse'].apply(lambda s: streetName(s))

dl=dl.drop(columns=['adresse'],axis=1)


dl=dl.rename(columns={'code':'Code Postal'})
dl['arr']=dl['Code Postal'].apply(lambda s: s[3:] if s[3]!='0' else s[4:])

In [19]:
dl


,ville,Code Postal,nom_restaurant,type,geometry,streetNumber,streetName,arr
0,PARIS,75002,SAINT-SAUVEUR,E,POINT (2.34948 48.86593),8,RUE SAINT-SAUVEUR,2
1,PARIS,75002,GRAMONT,E,POINT (2.33703 48.87096),28,RUE GRAMONT,2
2,PARIS,75013,TOLBIAC,E,POINT (2.35256 48.82603),166,RUE DE TOLBIAC,13
3,PARIS,75017,EPINETTES,S,POINT (2.32334 48.89642),51,RUE DES EPINETTES,17
4,PARIS,75010,VERDUN,E,POINT (2.36194 48.87627),14,AVENUE DE VERDUN,10
5,PARIS,75004,AVE MARIA,E,POINT (2.36053 48.85275),4,RUE DE L'AVE MARIA,4
6,PARIS,75008,MARCHE DE L'EUROPE,S,POINT (2.31345 48.87728),11,RUE MALLEVILLE,8
7,PARIS,75015,LA QUINTINIE,E,POINT (2.30718 48.83930),18,RUE BARGUE,15
8,PARIS,75003,AU MAIRE,E,POINT (2.35771 48.86445),2,RUE AU MAIRE,3
9,PARIS,75018,JOSEPH DE MAISTRE,E,POINT (2.33046 48.89289),88,RUE JOSEPH DE MAISTRE,18


Nous allons nous servir de cette table pour créer la variable nb_émeraudes, qui renseigne sur le nombre de restaurants émeraudes présents dans l'arrondissement où se situe chaque établissement de notre table

In [20]:
dl['arr'].value_counts()

18    4
5     4
14    4
13    4
15    3
12    3
9     2
2     2
17    2
3     2
20    2
4     2
11    2
10    2
19    1
16    1
6     1
7     1
8     1
Name: arr, dtype: int64

On peut maintenant créer notre variable nb_emeraudes : 


In [21]:
arr4=['18','13','5','14']                                # liste des arrondissements comportant 4 restaurants émeraudes
arr3=['12','15']
arr2=['20','4','10','17','3','11','2','9']
arr1=['8','6','16','19','7']
arr0=['1']

In [60]:
parisiens['nb_emeraudes']=parisiens['arr'].apply(lambda x: 4 if x in arr4 else 3 if x in arr3 else 2 if x in arr2 else 1 if x in arr1 else 0)

C:\Users\GUILLOT Robin\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [61]:
parisiens

,legalImmat,streetNumber,streetName,Code Postal,city,isHeadquarters,categoryType,legalName,contactName,contactFirstname,...,HT20,HT120,HT220,HT320,HT420,HT520,CPG20,RT20,arr,nb_emeraudes
10,304305519,5.0,AVENUE DU MAINE,75015,PARIS 15,1,NaN,LES CEDRES DU LIBAN,NaN,NaN,...,98.0,1.0,9.0,60.0,23.0,0.0,0.0,7.0,15,3
17,306332362,25.0,RUE DE LA HUCHETTE,75005,PARIS 5,1,NaN,LE SOUVLAKI ATHENIEN,NaN,NaN,...,84.0,1.0,14.0,35.0,24.0,0.0,0.0,3.0,5,4
19,307711895,15.0,PLACE DU TERTRE,75018,PARIS 18,1,NaN,LA CREMAILLERE DE MONTMARTRE,NaN,NaN,...,67.0,2.0,10.0,23.0,12.0,0.0,0.0,5.0,18,4
37,315946814,3.0,RUE DE L'ANCIENNE COMEDIE,75006,PARIS 6,1,NaN,F A S E P,NaN,NaN,...,105.0,1.0,10.0,40.0,33.0,7.0,0.0,4.0,6,1
51,318906591,11.0,RUE DE L ARRIVEE,75015,PARIS 15,0,NaN,LA BRIOCHE DOREE,NaN,NaN,...,98.0,1.0,9.0,60.0,23.0,0.0,0.0,7.0,15,3
57,319947537,14.0,RUE MABILLON,75006,PARIS 6,1,NaN,BRASSERIE DE LA GRILLE SAINT-GERMAIN,NaN,NaN,...,105.0,1.0,10.0,40.0,33.0,7.0,0.0,4.0,6,1
58,320228919,26.0,RUE PONCELET,75017,PARIS 17,1,NaN,AFO,NaN,NaN,...,128.0,2.0,9.0,61.0,33.0,1.0,0.0,3.0,17,2
85,325621605,14.0,BOULEVARD DE CLICHY,75018,PARIS 18,1,Commerçant,DRAGONAS*DENIS/,DRAGONAS,DENIS,...,67.0,2.0,10.0,23.0,12.0,0.0,0.0,5.0,18,4
97,327848875,7.0,RUE SAINTE APOLLINE,75003,PARIS 3,1,Commerçant,SALLES*MICHELE/,SALLES,MICHELE,...,31.0,0.0,4.0,9.0,9.0,2.0,0.0,0.0,3,2
121,331307975,110.0,RUE DE COURCELLES,75017,PARIS 17,1,NaN,COURCELLES-MEDERIC,NaN,NaN,...,128.0,2.0,9.0,61.0,33.0,1.0,0.0,3.0,17,2


## Scrapping : on va essayer de compléter les dataframes par :

- sur pages jaunes : prix moyen 

- sur www.societe.com : capital social 

- sur tripadvisor : note + nb d'avis (qui renseigne sur la frequentation) + prix


In [ ]:
!pip install bs4
!pip install selenium
!pip install msedge-selenium-tool 

In [2]:
import bs4
from urllib import request
import selenium
import html5lib

In [10]:
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome import options
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import ChromeOptions
from msedge.selenium_tools import Edge, EdgeOptions
from selenium.common.exceptions import NoSuchElementException

#### Récupération capital social sur societe.com

Nous récupérons les données de capital social pour les établissements en utilisant selenium. 

Pour l'exemple, en dessous on voit la sortie sur l'alogorithme appliqué sur 6000 établissements (ceux listés de 12000 à 18000)

In [12]:
options = webdriver.ChromeOptions()
options.add_argument("private")
options.add_argument("--start-maximized");
options.add_argument("--ignore-certificate-errors");
options.add_argument("--disable-popup-blocking");
options.add_argument("--incognito");
options.add_argument("--headless");



browser = webdriver.Chrome(executable_path="C:/Users/GUILLOT Robin/Documents/Robin Ensae/Matières/Python//chromedriver",options=options)

browser.get('https://www.societe.com/')

liste=parisiens['legalImmat'].iloc[12000:18000]
capital_social=[]

cookie=browser.find_element_by_id("didomi-notice-agree-button").click()

for siren in tqdm(liste):
    
    search_bar = browser.find_element_by_id("input_search")
    search_bar.send_keys(siren + Keys.ENTER)
    
    try:
        browser.find_element_by_xpath("//p[@class = 'red' and contains (text(),'Aucune réponse pour cette recherche')]")
        capital_social.append('NaN')
        continue
        
    except NoSuchElementException:
        pass
        
    try:
        cap_soc = browser.find_element_by_id('capital-histo-description').text
        capital_social.append(cap_soc)
        
    except NoSuchElementException:
        capital_social.append('NaN')
            
              
        
        
    
print(capital_social)

browser.quit() 


100%|████████████████████████████████████████████████████████████████████████████| 6000/6000 [5:40:10<00:00,  3.40s/it]


['8 000,00 €', '1 080 000,00 €', '842 490,00 €', '7 622,00 €', '7 622,45 €', '1 500,00 €', '10 400,00 €', '10 000,00 €', '10 000,00 €', '8 000,00 €', '20 000,00 €', '1 000,00 €', '2 500,00 €', '1 000,00 €', '3 000,00 €', '11 111,00 €', '6 000,00 €', '7 500,00 €', 'NaN', '3 000,00 €', '8 000,00 €', '2 000,00 €', '232 000,00 €', '343 800,00 €', '1 000,00 €', '1 000,00 €', '8 000,00 €', '6 000,00 €', '5 000,00 €', '90 000,00 €', '110 000,00 €', '10 000,00 €', '8 000,00 €', '4 000,00 €', '1 000,00 €', '2 000,00 €', '30 000,00 €', '1 000,00 €', '1 000 000,00 €', '2 000,00 €', 'NaN', '3 000,00 €', '1 500,00 €', '127 780,00 €', '1 000,00 €', '1 000,00 €', '10 000,00 €', '1 000,00 €', '2 500,00 €', '3 000,00 €', '10 000,00 €', '10 000,00 €', 'NaN', '76 420,00 €', '68 145,00 €', '500,00 €', '1 000,00 €', '5 000,00 €', '3 000,00 €', '5 000,00 €', '4 000,00 €', '10 000,00 €', '764 373,00 €', '10 000,00 €', '3 000,00 €', '10 000,00 €', '5 000,00 €', '2 000,00 €', '200 000,00 €', '5 000,00 €', '10 

Nous avons appliqué cet algortihme en plusieurs fois (c'est-à-dire pas d'un seul coup sur tous nos établissements parisiens), mais par tranche de 2000 voire 6000 (pour l'exemple du dessus).

A chaque fois nous stockons les listes obtenus dans un fichier, avec une instruction du type : 

In [203]:
with open('C:/Users/GUILLOT Robin/Documents/Robin Ensae/Matières/Python/Données/cap_social_12000_17999.txt', 'w') as f:
    for item in capital_social:
        f.write(str(item)+'\n')

Une fois que nous avons éxécuté notre algorithme sur tous les établissements, nous pouvons concaténer les listes stockées dans nos différents fichiers pour obtenir la liste finale du capital social sur l'ensemble de nos restaurants parisiens : 


In [41]:
with open('C:/Users/GUILLOT Robin/Documents/Robin Ensae/Matières/Python/Données/cap_social_0_1999.txt', 'r') as f:
    l1 = f.read().splitlines()

with open('C:/Users/GUILLOT Robin/Documents/Robin Ensae/Matières/Python/Données/cap_social_2000_3999.txt', 'r') as f:
    l2 = f.read().splitlines()

with open('C:/Users/GUILLOT Robin/Documents/Robin Ensae/Matières/Python/Données/cap_social_4000_5999.txt', 'r') as f:
    l3 = f.read().splitlines()

with open('C:/Users/GUILLOT Robin/Documents/Robin Ensae/Matières/Python/Données/cap_social_6000_7999.txt', 'r') as f:
    l4 = f.read().splitlines()

with open('C:/Users/GUILLOT Robin/Documents/Robin Ensae/Matières/Python/Données/cap_social_8000_9999.txt', 'r') as f:
    l5 = f.read().splitlines()
    
with open('C:/Users/GUILLOT Robin/Documents/Robin Ensae/Matières/Python/Données/cap_social_10000_11999.txt', 'r') as f:
    l6 = f.read().splitlines()
    
with open('C:/Users/GUILLOT Robin/Documents/Robin Ensae/Matières/Python/Données/cap_social_12000_17999.txt', 'r') as f:
    l7 = f.read().splitlines()
    
with open('C:/Users/GUILLOT Robin/Documents/Robin Ensae/Matières/Python/Données/cap_social_18000_22872.txt', 'r') as f:
    l8 = f.read().splitlines()

In [42]:
list_finale=l1+l2+l3+l4+l5+l6+l7+l8

In [43]:
len(list_finale)

22873

On peut maintenant concaténer list_finale à notre table parisiens.

Mais avant, on remet en forme nos données de capital social de sorte à avoir 
des valeurs numériques (donc le type doit être un flottant). 
Pour cela, on doit faire un certain nombre de petites opérations, parmi lesquelles supprimer le signe euro et les espaces.
On va donc appliquer une fonction transfo à notre série de données de capital_social

Pour cela, et après avoir passé en revue toutes nos données de capital_social, on a identifié certains types de cellules à prendre en compte:

In [44]:
# cellules qui contiennent la mention (minimum), assez présentes, qu'on doit supprimer dans notre fonction transfo
# Ex : 

print(list_finale[17938])


# cellules exprimées en Franc Suisse (deux en tout), on convertit en euros

print(list_finale[9122])
print(list_finale[20217])

list_finale[9122]='10230000'    # on garde une chaîne de caractères pour pouvoir appliquer notre fonction transfo() sans soucis
list_finale[20217]='10230000'


# cellules exprimées en Livre Sterling (deux en tout), on convertit en euros

print(list_finale[9322])
print(list_finale[15243])

list_finale[9322]='110'
list_finale[15243]='51366,7'

# cellule exprimée en US dollars (une en tout), on convertit en euros

print(list_finale[10037])

list_finale[10037]='830'

7 622,45 € (minimum)
11 000 000,00 Franc suisse
11 000 000,00 Franc suisse
100,00 Livre sterling
46 697,00 Livre sterling
1 000,00 Dollar US


In [62]:
# On écrit notre fonction transfo() (peut-être pas très optimisée en terme de manipulations d'expressions régulières...)

def transfo(s):
    e=re.sub('minimum','',s)
    f=re.sub('[(\s€)]','',e)
    g=re.sub(',','.',f)
    return float(g)

parisiens['Capital Social']=list_finale
parisiens['Capital Social']=parisiens['Capital Social'].apply(lambda s: transfo(s))

C:\Users\GUILLOT Robin\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\GUILLOT Robin\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [63]:
parisiens

,legalImmat,streetNumber,streetName,Code Postal,city,isHeadquarters,categoryType,legalName,contactName,contactFirstname,...,HT120,HT220,HT320,HT420,HT520,CPG20,RT20,arr,nb_emeraudes,Capital Social
10,304305519,5.0,AVENUE DU MAINE,75015,PARIS 15,1,NaN,LES CEDRES DU LIBAN,NaN,NaN,...,1.0,9.0,60.0,23.0,0.0,0.0,7.0,15,3,7622.45
17,306332362,25.0,RUE DE LA HUCHETTE,75005,PARIS 5,1,NaN,LE SOUVLAKI ATHENIEN,NaN,NaN,...,1.0,14.0,35.0,24.0,0.0,0.0,3.0,5,4,54881.65
19,307711895,15.0,PLACE DU TERTRE,75018,PARIS 18,1,NaN,LA CREMAILLERE DE MONTMARTRE,NaN,NaN,...,2.0,10.0,23.0,12.0,0.0,0.0,5.0,18,4,99220.80
37,315946814,3.0,RUE DE L'ANCIENNE COMEDIE,75006,PARIS 6,1,NaN,F A S E P,NaN,NaN,...,1.0,10.0,40.0,33.0,7.0,0.0,4.0,6,1,450000.00
51,318906591,11.0,RUE DE L ARRIVEE,75015,PARIS 15,0,NaN,LA BRIOCHE DOREE,NaN,NaN,...,1.0,9.0,60.0,23.0,0.0,0.0,7.0,15,3,95000000.00
57,319947537,14.0,RUE MABILLON,75006,PARIS 6,1,NaN,BRASSERIE DE LA GRILLE SAINT-GERMAIN,NaN,NaN,...,1.0,10.0,40.0,33.0,7.0,0.0,4.0,6,1,41700.00
58,320228919,26.0,RUE PONCELET,75017,PARIS 17,1,NaN,AFO,NaN,NaN,...,2.0,9.0,61.0,33.0,1.0,0.0,3.0,17,2,7622.45
85,325621605,14.0,BOULEVARD DE CLICHY,75018,PARIS 18,1,Commerçant,DRAGONAS*DENIS/,DRAGONAS,DENIS,...,2.0,10.0,23.0,12.0,0.0,0.0,5.0,18,4,NaN
97,327848875,7.0,RUE SAINTE APOLLINE,75003,PARIS 3,1,Commerçant,SALLES*MICHELE/,SALLES,MICHELE,...,0.0,4.0,9.0,9.0,2.0,0.0,0.0,3,2,NaN
121,331307975,110.0,RUE DE COURCELLES,75017,PARIS 17,1,NaN,COURCELLES-MEDERIC,NaN,NaN,...,2.0,9.0,61.0,33.0,1.0,0.0,3.0,17,2,7622.45


#### Récupération liens tripadvisor 

Nous récupérons les liens TripAdvisor des établissements en utilisant selenium.


In [ ]:
nombre_parisiens=len(parisiens)   # 22873

liens_trip_advisor=[]

for i in tqdm(range(nombre_parisiens)):  
    browser.get('https://www.google.fr/')
    search = browser.find_element_by_name('q')
    query = str(parisiens['legalName'][i]) + ' ' + 'restaurant paris tripadvisor'
    search.send_keys(query)
    search.send_keys(Keys.RETURN)
    
    try:
        browser.find_element(By.XPATH, '(//h3)[1]/../../a').click()
        liens_trip_advisor.append(browser.current_url)
        continue
    
    except NoSuchElementException:
        pass
   
    
# Certains restos ont eu comme lien le lien d'une review en particulier : 
# le code suivant permet de trouver la page globale de ces restos

 for i in tqdm(range(nombre_parisiens)): 
    if "https://www.tripadvisor.fr/ShowUserReviews" in liens_trip_advisor[i]:
        url = liens_trip_advisor[i]
        request_text = request.urlopen(url).read()
        page = bs4.BeautifulSoup(request_text, "lxml")        
        if page.find_all('span',{'class':'altHeadInline'}) != []:
            span = page.find('span',{'class':'altHeadInline'})
            if span.find('a')['href'] != []:
                href = span.find('a')['href']
                liens_trip_advisor[i] = 'https://www.tripadvisor.fr' + href
                


In [56]:
liens_trip_advisor

['https://www.tripadvisor.fr/Restaurant_Review-g187147-d1109794-Reviews-Les_Cedres_du_Liban-Paris_Ile_de_France.html',
 'https://www.tripadvisor.fr/Restaurant_Review-g187147-d3941770-Reviews-Souvlaki_Athenien-Paris_Ile_de_France.html',
 'https://www.tripadvisor.fr/Restaurant_Review-g187147-d1035528-Reviews-La_Cremaillere_1900-Paris_Ile_de_France.html',
 'https://www.tripadvisor.fr/Restaurants-g187147-Paris_Ile_de_France.html',
 'https://www.tripadvisor.fr/Restaurant_Review-g187147-d2229107-Reviews-Brioche_Doree-Paris_Ile_de_France.html',
 'https://www.tripadvisor.fr/Restaurant_Review-g187147-d718680-Reviews-La_Grille_Vintage-Paris_Ile_de_France.html',
 'https://www.google.fr/search?source=hp&ei=bV7OX-nYMoSBjLsPt4WA0AM&q=AFO+restaurant+paris+tripadvisor&oq=AFO+restaurant+paris+tripadvisor&gs_lcp=CgZwc3ktYWIQAzIICCEQFhAdEB5Q2AZYiQxggA1oAHAAeACAAfIBiAGhBJIBBTEuMC4ymAEAoAEBqgEHZ3dzLXdpeg&sclient=psy-ab&ved=0ahUKEwipw_2Cq7ztAhWEAGMBHbcCADoQ4dUDCAw&uact=5',
 'https://www.google.fr/search?sou

In [64]:
parisiens['Lien TripAdvisor']=liens_trip_advisor

C:\Users\GUILLOT Robin\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [65]:
parisiens

,legalImmat,streetNumber,streetName,Code Postal,city,isHeadquarters,categoryType,legalName,contactName,contactFirstname,...,HT220,HT320,HT420,HT520,CPG20,RT20,arr,nb_emeraudes,Capital Social,Lien TripAdvisor
10,304305519,5.0,AVENUE DU MAINE,75015,PARIS 15,1,NaN,LES CEDRES DU LIBAN,NaN,NaN,...,9.0,60.0,23.0,0.0,0.0,7.0,15,3,7622.45,https://www.tripadvisor.fr/Restaurant_Review-g...
17,306332362,25.0,RUE DE LA HUCHETTE,75005,PARIS 5,1,NaN,LE SOUVLAKI ATHENIEN,NaN,NaN,...,14.0,35.0,24.0,0.0,0.0,3.0,5,4,54881.65,https://www.tripadvisor.fr/Restaurant_Review-g...
19,307711895,15.0,PLACE DU TERTRE,75018,PARIS 18,1,NaN,LA CREMAILLERE DE MONTMARTRE,NaN,NaN,...,10.0,23.0,12.0,0.0,0.0,5.0,18,4,99220.80,https://www.tripadvisor.fr/Restaurant_Review-g...
37,315946814,3.0,RUE DE L'ANCIENNE COMEDIE,75006,PARIS 6,1,NaN,F A S E P,NaN,NaN,...,10.0,40.0,33.0,7.0,0.0,4.0,6,1,450000.00,https://www.tripadvisor.fr/Restaurants-g187147...
51,318906591,11.0,RUE DE L ARRIVEE,75015,PARIS 15,0,NaN,LA BRIOCHE DOREE,NaN,NaN,...,9.0,60.0,23.0,0.0,0.0,7.0,15,3,95000000.00,https://www.tripadvisor.fr/Restaurant_Review-g...
57,319947537,14.0,RUE MABILLON,75006,PARIS 6,1,NaN,BRASSERIE DE LA GRILLE SAINT-GERMAIN,NaN,NaN,...,10.0,40.0,33.0,7.0,0.0,4.0,6,1,41700.00,https://www.tripadvisor.fr/Restaurant_Review-g...
58,320228919,26.0,RUE PONCELET,75017,PARIS 17,1,NaN,AFO,NaN,NaN,...,9.0,61.0,33.0,1.0,0.0,3.0,17,2,7622.45,https://www.google.fr/search?source=hp&ei=bV7O...
85,325621605,14.0,BOULEVARD DE CLICHY,75018,PARIS 18,1,Commerçant,DRAGONAS*DENIS/,DRAGONAS,DENIS,...,10.0,23.0,12.0,0.0,0.0,5.0,18,4,NaN,https://www.google.fr/search?source=hp&ei=cl7O...
97,327848875,7.0,RUE SAINTE APOLLINE,75003,PARIS 3,1,Commerçant,SALLES*MICHELE/,SALLES,MICHELE,...,4.0,9.0,9.0,2.0,0.0,0.0,3,2,NaN,https://www.tripadvisor.fr/Restaurant_Review-g...
121,331307975,110.0,RUE DE COURCELLES,75017,PARIS 17,1,NaN,COURCELLES-MEDERIC,NaN,NaN,...,9.0,61.0,33.0,1.0,0.0,3.0,17,2,7622.45,https://www.tripadvisor.fr/Hotel_Review-g18714...


In [66]:
#On ne garde que les restaurants dont le lien est bien une page tripadvisor : on en a 14 000 environ

parisiens= parisiens[parisiens['Lien TripAdvisor'].str.contains('https://www.tripadvisor.fr/Restaurant_Review')]

In [67]:
parisiens

,legalImmat,streetNumber,streetName,Code Postal,city,isHeadquarters,categoryType,legalName,contactName,contactFirstname,...,HT220,HT320,HT420,HT520,CPG20,RT20,arr,nb_emeraudes,Capital Social,Lien TripAdvisor
10,304305519,5.0,AVENUE DU MAINE,75015,PARIS 15,1,NaN,LES CEDRES DU LIBAN,NaN,NaN,...,9.0,60.0,23.0,0.0,0.0,7.0,15,3,7622.45,https://www.tripadvisor.fr/Restaurant_Review-g...
17,306332362,25.0,RUE DE LA HUCHETTE,75005,PARIS 5,1,NaN,LE SOUVLAKI ATHENIEN,NaN,NaN,...,14.0,35.0,24.0,0.0,0.0,3.0,5,4,54881.65,https://www.tripadvisor.fr/Restaurant_Review-g...
19,307711895,15.0,PLACE DU TERTRE,75018,PARIS 18,1,NaN,LA CREMAILLERE DE MONTMARTRE,NaN,NaN,...,10.0,23.0,12.0,0.0,0.0,5.0,18,4,99220.80,https://www.tripadvisor.fr/Restaurant_Review-g...
51,318906591,11.0,RUE DE L ARRIVEE,75015,PARIS 15,0,NaN,LA BRIOCHE DOREE,NaN,NaN,...,9.0,60.0,23.0,0.0,0.0,7.0,15,3,95000000.00,https://www.tripadvisor.fr/Restaurant_Review-g...
57,319947537,14.0,RUE MABILLON,75006,PARIS 6,1,NaN,BRASSERIE DE LA GRILLE SAINT-GERMAIN,NaN,NaN,...,10.0,40.0,33.0,7.0,0.0,4.0,6,1,41700.00,https://www.tripadvisor.fr/Restaurant_Review-g...
97,327848875,7.0,RUE SAINTE APOLLINE,75003,PARIS 3,1,Commerçant,SALLES*MICHELE/,SALLES,MICHELE,...,4.0,9.0,9.0,2.0,0.0,0.0,3,2,NaN,https://www.tripadvisor.fr/Restaurant_Review-g...
124,331831826,87.0,RUE DE TURBIGO,75003,PARIS 3,1,NaN,SOC LE TEMPLE,NaN,NaN,...,4.0,9.0,9.0,2.0,0.0,0.0,3,2,7622.45,https://www.tripadvisor.fr/Restaurant_Review-g...
191,343281945,13.0,RUE DES COURONNES,75020,PARIS 20,1,Commerçant,ZIZI*LOFTI/,ZIZI,LOFTI,...,4.0,12.0,0.0,0.0,0.0,3.0,20,2,NaN,https://www.tripadvisor.fr/Restaurant_Review-g...
227,349664664,127.0,RUE AMELOT,75011,PARIS 11,1,Commerçant,DE AZEVEDO PEREIRA*JOSE LUIS/,DE AZEVEDO PEREIRA,JOSE,...,12.0,33.0,12.0,2.0,0.0,4.0,11,2,NaN,https://www.tripadvisor.fr/Restaurant_Review-g...
237,350568242,73.0,BOULEVARD SAINT GERMAIN,75005,PARIS 5,1,NaN,LA SIRENE,NaN,NaN,...,14.0,35.0,24.0,0.0,0.0,3.0,5,4,7622.45,https://www.tripadvisor.fr/Restaurant_Review-g...


In [ ]:
parisiens.reset_index(drop = True, inplace = True)

#### Récupération sur TripAdvisor des notes, nombre d'avis et prix


##### On utilise cette fois-ci BeautifulSoup (cela est suffisant, puisqu'on a nos liens TripAdvisor)
Pour chaque lien, on va récupérer les informations du restaurant associé, plus précisément 10 informations en tout, stockées dans dans 9 listes : 

- l1 : Style de nourriture
- l2 : Note Globale
- l3 : Catégorie de prix
- l4 : Fourchette de prix
- l5 : Note de cuisine
- l6 : Note de service
- l7 : Note qualité/prix
- l8 : Note d'ambiance
- l9 : Nombre d'avis
- l10 : code postal du restaurant (variable de contrôle pour vérifier que les bons restaurants sont scrappés, mais ne sert pas pour la suite)

Pour l'exemple, en-dessous on voit la sortie de l'algorithme pour 500 restaurants environ (de 7896 à 8408) : 

In [ ]:
l1=[]
l2=[]
l3=[]
l4=[]
l5=[]
l6=[]
l7=[]
l8=[]
l9=[]
l10=[]


for i in tqdm(range(7896,8408)):         
    url = liens_trip_advisor[i]                      
    req=request.Request(url)
    webpage = request.urlopen(req).read()
    page = bs4.BeautifulSoup(webpage, "lxml")

    adress=page.find('a',{"href":"#MAPVIEW"})
    try:
        l10.append(re.sub("[^0-9]", "",str(adress.contents))[-5:])
    
    except AttributeError:
         l10.append("NR")
    
    divs = page.find('div',{'class':'page'})
    try:
        infos = divs.find_all('a')
        
        try: 
            l1.append(infos[6].contents)
            
        except TypeError:
            l1.append("NR")
        
        try:
            l2.append(infos[1].contents[0]['title'][0:3])
            
        except TypeError:
            l2.append("NR")
    
        try:
            l3.append(infos[3].contents)
        
        except:
            l3.append("NR")
            
    except AttributeError:
        l1.append("NR")
        l2.append("NR")
        l3.append("NR")
    
            
    infos2 = page.find_all('div',{"class":"_1XLfiSsv"})
    try:
        l4.append(infos2[0].contents[0])
    
    except IndexError:
        l4.append("NR") 

    infos3 = page.find_all('span',{"class":"_377onWB-"})
    if len(infos3)==0:
        l5.append("NR")
        l6.append("NR")
        l7.append("NR")
        l8.append("NR")
        
    if len(infos3)==1:
        l5.append(int(infos3[0].contents[0]['class'][1][7:])/10)
        l6.append("NR")
        l7.append("NR")
        l8.append("NR")
        
    if len(infos3) == 2:
        l5.append(int(infos3[0].contents[0]['class'][1][7:])/10)
        l6.append(int(infos3[1].contents[0]['class'][1][7:])/10)
        l7.append("NR")
        l8.append("NR")
        
        
    if len(infos3) == 3:
        l5.append(int(infos3[0].contents[0]['class'][1][7:])/10)
        l6.append(int(infos3[1].contents[0]['class'][1][7:])/10)
        l7.append(int(infos3[2].contents[0]['class'][1][7:])/10)
        l8.append("NR")
      
    if len(infos3) == 4:
        l5.append(int(infos3[0].contents[0]['class'][1][7:])/10)
        l6.append(int(infos3[1].contents[0]['class'][1][7:])/10)
        l7.append(int(infos3[2].contents[0]['class'][1][7:])/10)
        l8.append(int(infos3[3].contents[0]['class'][1][7:])/10)
    
    if len(infos3) >= 5:
        l5.append("NR")
        l6.append("NR")
        l7.append("NR")
        l8.append("NR")
        
    try:
        l9.append(re.sub("[^0-9]", "", page.find('span',{"class":"_3Wub8auF"}).text))
    except AttributeError:
        l9.append("NR")
   
 # on rajoute une condition d'arrêt : en effet, TripAdvisor nous bloque au bout de 500 itérations environ, ce qui fait que 
 # la page qu'on lui demande de requêter est vide, et l'algorithme va alors mettre que des NaN à notre liste
 # Ainsi, lorsque pour un restaurant, toutes les informations ajoutées à nos listes sont sur "NR", c'est que TripAdvicos 
 # nous a bloqués temporairement, et alors on arrête l'algorithme, et on recommence là où on s'est arrêtés
    j=i
    if l1[j]==l2[j]==l3[j]==l4[j]==l5[j]==l6[j]==l7[j]==l8[j]==l9[j]==l10[j]=="NR":
        break

In [18]:
print('Style de nourriture',l1)
print('--------------------------------------')
print('\n')
print('Note Globale',l2)
print('--------------------------------------')
print('\n')
print('Catégorie de prix',l3)
print('--------------------------------------')
print('\n')
print('Fourchette de prix',l4)
print('--------------------------------------')
print('\n')
print('Note de cuisine',l5)
print('--------------------------------------')
print('\n')
print('Note de service',l6)
print('--------------------------------------')
print('\n')
print('Note qualité-prix',l7)
print('--------------------------------------')
print('\n')
print('Note ambiance',l8)
print('--------------------------------------')
print('\n')
print('Nombre avis',l9)
print('--------------------------------------')
print('\n')
print('Code Postal',l10)
print('--------------------------------------')
print('\n')

Style de nourriture ["['Européenne']", "['Européenne']", "['27 rue Rodier, 75009 Paris France']", "['Café']", "['+ Ajouter le numéro de téléphone']", "['Plats sans gluten']", "['Méditerranéenne']", "['Asiatique']", "['Végétariens bienvenus']", "['Végétariens bienvenus']", "['Asiatique']", "['Shanghai']", "['19 Boulevard Garibaldi, 75015, 75015 Paris France']", "['27 rue Salneuve, 75017 Paris France']", "['+33 1 43 43 64 55']", "['Méditerranéenne']", "['Européenne']", "['10 Rue Mandar, 75002 Paris France']", "['Végétariens bienvenus']", "['Végétariens bienvenus']", "['Végétariens bienvenus']", "['Géorgienne']", "['Napolitaine']", "['+ Ajouter le site web']", "['Fruits de mer & Poisson']", '["62 rue d\'Orsel, 75018 Paris France"]', "['Bars-restaurants']", "['Diner']", "['+33 1 44 26 34 11']", "['+ Ajouter horaires']", "['10 Rue Du Cardinal Lemoine, 75005 Paris France']", "['+33 6 84 81 43 32']", "['+ Ajouter le site web']", '["Europe de l\'Est"]', "['Asiatique']", "['Vietnamienne']", "['

On concatène les listes finales obtenues à nottre dataframe parisiens (tous les détails ne sont pas mis ici), et on obtient finalement le dataframe suivant : 

In [19]:
parisiens=pd.read_csv("C:/Users/GUILLOT Robin/Documents/Robin Ensae/Matières/Python/semivictoire.csv",sep=";")

In [20]:
parisiens

,legalImmat,streetNumber,streetName,Code Postal,city,isHeadquarters,categoryType,legalName,contactName,contactFirstname,...,Lien TripAdvisor,Style de nourriture,Note Globale,Catégorie de prix,Fourchette de prix,Note de cuisine,Note de service,Note qualité-prix,Note ambiance,Nombre avis
0,304305519,5.0,AVENUE DU MAINE,75015,PARIS 15,1,NaN,LES CEDRES DU LIBAN,NaN,NaN,...,https://www.tripadvisor.fr/Restaurant_Review-g...,['Saine'],"4,5",['€€-€€€'],8€ - 21€,4.5,4.5,4.0,3.5,850
1,306332362,25.0,RUE DE LA HUCHETTE,75005,PARIS 5,1,NaN,LE SOUVLAKI ATHENIEN,NaN,NaN,...,https://www.tripadvisor.fr/Restaurant_Review-g...,['+33 1 43 54 96 95'],"4,0",['€'],4€ - 15€,4.0,4.0,4.0,4.0,59
2,307711895,15.0,PLACE DU TERTRE,75018,PARIS 18,1,NaN,LA CREMAILLERE DE MONTMARTRE,NaN,NaN,...,https://www.tripadvisor.fr/Restaurant_Review-g...,"['15 Place du Tertre, 75018 Paris France']","3,5",['€€-€€€'],17€ - 37€,3.5,3.5,3.5,4.0,1036
3,318906591,11.0,RUE DE L ARRIVEE,75015,PARIS 15,0,NaN,LA BRIOCHE DOREE,NaN,NaN,...,https://www.tripadvisor.fr/Restaurant_Review-g...,['Restauration rapide'],"4,0",['€'],"Française, Café, Restauration rapide, Européenne",4.0,3.5,4.0,3.5,421
4,319947537,14.0,RUE MABILLON,75006,PARIS 6,1,NaN,BRASSERIE DE LA GRILLE SAINT-GERMAIN,NaN,NaN,...,https://www.tripadvisor.fr/Restaurant_Review-g...,"['14, rue Mabillon, 75006 Paris France']","3,0",['€€-€€€'],"Française, Européenne",3.0,3.5,3.0,3.5,336
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14825,823268370,17.0,RUE VIGNON,75008,PARIS 8,1,NaN,HU JIN,NaN,NaN,...,https://www.tripadvisor.fr/Restaurant_Review-g...,['Asiatique'],"4,0",['€€€€'],"Japonaise, Sushi, Asiatique",4.5,4.5,3.5,3.5,167
14826,823628706,115.0,BOULEVARD DAVOUT,75020,PARIS 20,1,NaN,YASMINE D'ORIENT,NaN,NaN,...,https://www.tripadvisor.fr/Restaurant_Review-g...,['Végétariens bienvenus'],"4,0",['€'],"Indienne, Pakistanaise",4.0,4.0,4.0,4.0,288
14827,823812573,248.0,RUE DU FBG ST ANTOINE,75012,PARIS 12,1,NaN,CHEZ XIA,NaN,NaN,...,https://www.tripadvisor.fr/Restaurant_Review-g...,"[""20 Rue de Bellechasse Musee d'Orsay, 7eme, 7...","2,5",['€'],"Chinoise, Asiatique",3.0,3.0,3.0,NR,27
14828,824340020,12.0,RUE LAKANAL,75015,PARIS 15,1,NaN,SMR,NaN,NaN,...,https://www.tripadvisor.fr/Restaurant_Review-g...,['Saine'],"4,0",['€€€€'],28€ - 50€,4.0,4.5,3.5,4.5,319


Recodage de certaines variables et nettoyage des sorties : on va 


1)Nettoyer la variable style de nourriture qui contient des sorties diverses et variées, comme on peut le voir au-dessus

2)Recoder la variable catégorie de prix en plusieurs modalités ("faible","moyen","élevé") selon le nombre de signe "euros"

3)Nettoyer et transformer la variable fourchette de prix en deux variables : prix minimum et prix maximum

4)Remplacer tous les Non Renseigné/NR par des NaN pour que ce soit uniformisé sur notre Dataframe

1 : Variable style de nourriture

Il s'agit plus d'une information pour compléter qui ne va pas tant nous servir...

In [107]:
args_acceptes=["Végétariens bienvenus","Espagnole","Européenne","Moyen-Orient","Cantonaise","Sicilienne"
               ,"Coréenne","Asiatique","Saine","Tunisienne","Vietnamienne","Japonaise","Thaï","Américaine","Méditerranéenne",
               "Grecque","Turque","Restauration rapide","cambodgienne"]

In [108]:
parisiens

,legalImmat,streetNumber,streetName,Code Postal,city,isHeadquarters,categoryType,legalName,contactName,contactFirstname,...,Lien TripAdvisor,Style de nourriture,Note Globale,Catégorie de prix,Fourchette de prix,Note de cuisine,Note de service,Note qualité-prix,Note ambiance,Nombre avis
0,304305519,5.0,AVENUE DU MAINE,75015,PARIS 15,1,NaN,LES CEDRES DU LIBAN,NaN,NaN,...,https://www.tripadvisor.fr/Restaurant_Review-g...,['Saine'],"4,5",['€€-€€€'],8€ - 21€,4.5,4.5,4.0,3.5,850
1,306332362,25.0,RUE DE LA HUCHETTE,75005,PARIS 5,1,NaN,LE SOUVLAKI ATHENIEN,NaN,NaN,...,https://www.tripadvisor.fr/Restaurant_Review-g...,['+33 1 43 54 96 95'],"4,0",['€'],4€ - 15€,4.0,4.0,4.0,4.0,59
2,307711895,15.0,PLACE DU TERTRE,75018,PARIS 18,1,NaN,LA CREMAILLERE DE MONTMARTRE,NaN,NaN,...,https://www.tripadvisor.fr/Restaurant_Review-g...,"['15 Place du Tertre, 75018 Paris France']","3,5",['€€-€€€'],17€ - 37€,3.5,3.5,3.5,4.0,1036
3,318906591,11.0,RUE DE L ARRIVEE,75015,PARIS 15,0,NaN,LA BRIOCHE DOREE,NaN,NaN,...,https://www.tripadvisor.fr/Restaurant_Review-g...,['Restauration rapide'],"4,0",['€'],"Française, Café, Restauration rapide, Européenne",4.0,3.5,4.0,3.5,421
4,319947537,14.0,RUE MABILLON,75006,PARIS 6,1,NaN,BRASSERIE DE LA GRILLE SAINT-GERMAIN,NaN,NaN,...,https://www.tripadvisor.fr/Restaurant_Review-g...,"['14, rue Mabillon, 75006 Paris France']","3,0",['€€-€€€'],"Française, Européenne",3.0,3.5,3.0,3.5,336
5,327848875,7.0,RUE SAINTE APOLLINE,75003,PARIS 3,1,Commerçant,SALLES*MICHELE/,SALLES,MICHELE,...,https://www.tripadvisor.fr/Restaurant_Review-g...,['Saine'],"5,0",['€€-€€€'],4€ - 13€,5.0,5.0,5.0,Non renseigné,25
6,331831826,87.0,RUE DE TURBIGO,75003,PARIS 3,1,NaN,SOC LE TEMPLE,NaN,NaN,...,https://www.tripadvisor.fr/Restaurant_Review-g...,['Café'],"4,5",['€'],"Française, Café, Européenne",4.5,4.5,4.5,Non renseigné,24
7,343281945,13.0,RUE DES COURONNES,75020,PARIS 20,1,Commerçant,ZIZI*LOFTI/,ZIZI,LOFTI,...,https://www.tripadvisor.fr/Restaurant_Review-g...,['Végétariens bienvenus'],"4,0",['€€-€€€'],10€ - 22€,4.0,4.0,4.0,4.0,364
8,349664664,127.0,RUE AMELOT,75011,PARIS 11,1,Commerçant,DE AZEVEDO PEREIRA*JOSE LUIS/,DE AZEVEDO PEREIRA,JOSE,...,https://www.tripadvisor.fr/Restaurant_Review-g...,['Espagnole'],"4,0",['€€-€€€'],"Méditerranéenne, Européenne, Espagnole",4.0,3.5,4.0,4.0,115
9,350568242,73.0,BOULEVARD SAINT GERMAIN,75005,PARIS 5,1,NaN,LA SIRENE,NaN,NaN,...,https://www.tripadvisor.fr/Restaurant_Review-g...,['+33 1 46 36 07 46'],"3,5",['€€-€€€'],Italienne,4.0,4.5,4.0,Non renseigné,26


In [109]:
parisiens["Style de nourriture"]=parisiens["Style de nourriture"].apply(lambda s: re.sub('[\[\]\']','',s))
parisiens["Style de nourriture"]=parisiens["Style de nourriture"].apply(lambda s: s if s in args_acceptes else 'NaN')

2 : Variable catégorie de prix

Il s'agit d'une information inmportante, quasiment toujours renseignée

Tripadvisor a 3 niveaux échelles : 

'€' : prix faible

'€€-€€€' : prix intermédiaire

'€€€€' : prix élevé

In [110]:
def transfo(s):
    if s.find('€')!=-1:
        s=re.sub('[\'-\[\]]','',s)
        if len(s)==1:
            s='faible'
        elif len(s)==4:
            s='élevé'
        else:
            s='moyen'
    
    else:
        s='NaN'
    
    return s
        
    

In [111]:
parisiens["Catégorie de prix"]=parisiens["Catégorie de prix"].apply(lambda s: transfo(s))

3 : Variable fourchettede prix

Variable également importante, moins renseignée que la précédente.

On nettoie et on crée les 3 variables : prix inf, prix sup, prix moyen


In [113]:
def transfobis(s):
    prix_inf=0
    prix_sup=0
    prix_moyen=0
    if s.find('€')==-1:
        prix_inf='NaN'
        prix_sup='NaN'
        prix_moyen='NaN'
    else:
        b=re.sub('[\s€]','',s)
        c=b.split('-')
        prix_inf=float(c[0])
        prix_sup=float(c[1])
        prix_moyen=(prix_inf+prix_sup)/2
    
    return prix_inf,prix_sup,prix_moyen
        
            

In [114]:
parisiens["Prix inf"]=parisiens["Fourchette de prix"].apply(lambda s: transfobis(s)[0])
parisiens["Prix sup"]=parisiens["Fourchette de prix"].apply(lambda s: transfobis(s)[1])
parisiens["Prix moyen"]=parisiens["Fourchette de prix"].apply(lambda s: transfobis(s)[2])
parisiens=parisiens.drop(columns=["Fourchette de prix"],axis=1)

In [115]:
parisiens=parisiens.astype({'Prix inf': float, 'Prix sup': float, 'Prix moyen':float})

In [116]:
parisiens


,legalImmat,streetNumber,streetName,Code Postal,city,isHeadquarters,categoryType,legalName,contactName,contactFirstname,...,Note Globale,Catégorie de prix,Note de cuisine,Note de service,Note qualité-prix,Note ambiance,Nombre avis,Prix inf,Prix sup,Prix moyen
0,304305519,5.0,AVENUE DU MAINE,75015,PARIS 15,1,NaN,LES CEDRES DU LIBAN,NaN,NaN,...,"4,5",moyen,4.5,4.5,4.0,3.5,850,8.0,21.0,14.5
1,306332362,25.0,RUE DE LA HUCHETTE,75005,PARIS 5,1,NaN,LE SOUVLAKI ATHENIEN,NaN,NaN,...,"4,0",faible,4.0,4.0,4.0,4.0,59,4.0,15.0,9.5
2,307711895,15.0,PLACE DU TERTRE,75018,PARIS 18,1,NaN,LA CREMAILLERE DE MONTMARTRE,NaN,NaN,...,"3,5",moyen,3.5,3.5,3.5,4.0,1036,17.0,37.0,27.0
3,318906591,11.0,RUE DE L ARRIVEE,75015,PARIS 15,0,NaN,LA BRIOCHE DOREE,NaN,NaN,...,"4,0",faible,4.0,3.5,4.0,3.5,421,NaN,NaN,NaN
4,319947537,14.0,RUE MABILLON,75006,PARIS 6,1,NaN,BRASSERIE DE LA GRILLE SAINT-GERMAIN,NaN,NaN,...,"3,0",moyen,3.0,3.5,3.0,3.5,336,NaN,NaN,NaN
5,327848875,7.0,RUE SAINTE APOLLINE,75003,PARIS 3,1,Commerçant,SALLES*MICHELE/,SALLES,MICHELE,...,"5,0",moyen,5.0,5.0,5.0,Non renseigné,25,4.0,13.0,8.5
6,331831826,87.0,RUE DE TURBIGO,75003,PARIS 3,1,NaN,SOC LE TEMPLE,NaN,NaN,...,"4,5",faible,4.5,4.5,4.5,Non renseigné,24,NaN,NaN,NaN
7,343281945,13.0,RUE DES COURONNES,75020,PARIS 20,1,Commerçant,ZIZI*LOFTI/,ZIZI,LOFTI,...,"4,0",moyen,4.0,4.0,4.0,4.0,364,10.0,22.0,16.0
8,349664664,127.0,RUE AMELOT,75011,PARIS 11,1,Commerçant,DE AZEVEDO PEREIRA*JOSE LUIS/,DE AZEVEDO PEREIRA,JOSE,...,"4,0",moyen,4.0,3.5,4.0,4.0,115,NaN,NaN,NaN
9,350568242,73.0,BOULEVARD SAINT GERMAIN,75005,PARIS 5,1,NaN,LA SIRENE,NaN,NaN,...,"3,5",moyen,4.0,4.5,4.0,Non renseigné,26,NaN,NaN,NaN


4 : On remplace tous les NR/Non renseignés par des NaN sur nos variables de notes


In [117]:
def transfoter(s):
    list=['Non renseigné','NR']
    if s in list:
        s='NaN'
    return s

In [118]:
parisiens["Note Globale"]=parisiens["Note Globale"].apply(lambda s: transfoter(s))
parisiens["Nombre avis"]=parisiens["Nombre avis"].apply(lambda s: transfoter(s))
parisiens["Note de cuisine"]=parisiens["Note de cuisine"].apply(lambda s: transfoter(s))
parisiens["Note de service"]=parisiens["Note de service"].apply(lambda s: transfoter(s))
parisiens["Note qualité-prix"]=parisiens["Note qualité-prix"].apply(lambda s: transfoter(s))
parisiens["Note ambiance"]=parisiens["Note ambiance"].apply(lambda s: transfoter(s))

In [119]:
# on transforme la variable Note Globale en valeurs numériques

parisiens["Note Globale"]=parisiens["Note Globale"].apply(lambda s: float(re.sub(',','.',str(s))))

In [120]:
parisiens

,legalImmat,streetNumber,streetName,Code Postal,city,isHeadquarters,categoryType,legalName,contactName,contactFirstname,...,Note Globale,Catégorie de prix,Note de cuisine,Note de service,Note qualité-prix,Note ambiance,Nombre avis,Prix inf,Prix sup,Prix moyen
0,304305519,5.0,AVENUE DU MAINE,75015,PARIS 15,1,NaN,LES CEDRES DU LIBAN,NaN,NaN,...,4.5,moyen,4.5,4.5,4.0,3.5,850,8.0,21.0,14.5
1,306332362,25.0,RUE DE LA HUCHETTE,75005,PARIS 5,1,NaN,LE SOUVLAKI ATHENIEN,NaN,NaN,...,4.0,faible,4.0,4.0,4.0,4.0,59,4.0,15.0,9.5
2,307711895,15.0,PLACE DU TERTRE,75018,PARIS 18,1,NaN,LA CREMAILLERE DE MONTMARTRE,NaN,NaN,...,3.5,moyen,3.5,3.5,3.5,4.0,1036,17.0,37.0,27.0
3,318906591,11.0,RUE DE L ARRIVEE,75015,PARIS 15,0,NaN,LA BRIOCHE DOREE,NaN,NaN,...,4.0,faible,4.0,3.5,4.0,3.5,421,NaN,NaN,NaN
4,319947537,14.0,RUE MABILLON,75006,PARIS 6,1,NaN,BRASSERIE DE LA GRILLE SAINT-GERMAIN,NaN,NaN,...,3.0,moyen,3.0,3.5,3.0,3.5,336,NaN,NaN,NaN
5,327848875,7.0,RUE SAINTE APOLLINE,75003,PARIS 3,1,Commerçant,SALLES*MICHELE/,SALLES,MICHELE,...,5.0,moyen,5.0,5.0,5.0,NaN,25,4.0,13.0,8.5
6,331831826,87.0,RUE DE TURBIGO,75003,PARIS 3,1,NaN,SOC LE TEMPLE,NaN,NaN,...,4.5,faible,4.5,4.5,4.5,NaN,24,NaN,NaN,NaN
7,343281945,13.0,RUE DES COURONNES,75020,PARIS 20,1,Commerçant,ZIZI*LOFTI/,ZIZI,LOFTI,...,4.0,moyen,4.0,4.0,4.0,4.0,364,10.0,22.0,16.0
8,349664664,127.0,RUE AMELOT,75011,PARIS 11,1,Commerçant,DE AZEVEDO PEREIRA*JOSE LUIS/,DE AZEVEDO PEREIRA,JOSE,...,4.0,moyen,4.0,3.5,4.0,4.0,115,NaN,NaN,NaN
9,350568242,73.0,BOULEVARD SAINT GERMAIN,75005,PARIS 5,1,NaN,LA SIRENE,NaN,NaN,...,3.5,moyen,4.0,4.5,4.0,NaN,26,NaN,NaN,NaN


On ne va sélectionner ici que les restaurants. En fait, on a ici une table plus grosse qui contient les commerçants, artisans... et même parfois des établissements de soin qui n'ont rien à voir avec des restaurants...

On ne va donc garder dans la table précédente que ceux dont le "categoryType" est NaN, c'est-à-dire ceux qui ne sont pas commerçants. 
Pour ceux-là d'ailleurs, l'établissement porte leur nom : il est donc très difficile d'obtenir plus de données que ça sur ces établissements, et ce sont ceux qui correspondent aux lignes où souvent le nombre d'avis ou prix moyen n'est pas renseigné.

D'ailleurs, si on regarde seulement pour les commerçants, on voit que le capital social n'est jamais renseigné pour eux (ce qui est normal, et alors qu'il l'est quasiment pour tous les autres)

Bref, et on aurait pu le faire avant tous nos scrappings, mais on enlève les commerçants et artisans de notre table parisiens, et donc du même coup les colonnes contactName et contactFirstName qui ne sont renseignées que pour les commerçants

In [121]:
parisiens['categoryType'].fillna(0, inplace = True)
parisiens=parisiens[parisiens['categoryType']==0]

parisiens=parisiens.drop(columns=['categoryType','contactName','contactFirstname'],axis=1)

In [122]:
parisiens

,legalImmat,streetNumber,streetName,Code Postal,city,isHeadquarters,legalName,legalStructure,employeesNb,categorySize,...,Note Globale,Catégorie de prix,Note de cuisine,Note de service,Note qualité-prix,Note ambiance,Nombre avis,Prix inf,Prix sup,Prix moyen
0,304305519,5.0,AVENUE DU MAINE,75015,PARIS 15,1,LES CEDRES DU LIBAN,SARL,6.0,PME,...,4.5,moyen,4.5,4.5,4.0,3.5,850,8.0,21.0,14.5
1,306332362,25.0,RUE DE LA HUCHETTE,75005,PARIS 5,1,LE SOUVLAKI ATHENIEN,SARL,0.0,PME,...,4.0,faible,4.0,4.0,4.0,4.0,59,4.0,15.0,9.5
2,307711895,15.0,PLACE DU TERTRE,75018,PARIS 18,1,LA CREMAILLERE DE MONTMARTRE,SAS,20.0,PME,...,3.5,moyen,3.5,3.5,3.5,4.0,1036,17.0,37.0,27.0
3,318906591,11.0,RUE DE L ARRIVEE,75015,PARIS 15,0,LA BRIOCHE DOREE,SAS,1900.0,ETI,...,4.0,faible,4.0,3.5,4.0,3.5,421,NaN,NaN,NaN
4,319947537,14.0,RUE MABILLON,75006,PARIS 6,1,BRASSERIE DE LA GRILLE SAINT-GERMAIN,SAS,18.0,PME,...,3.0,moyen,3.0,3.5,3.0,3.5,336,NaN,NaN,NaN
6,331831826,87.0,RUE DE TURBIGO,75003,PARIS 3,1,SOC LE TEMPLE,SARL,5.0,PME,...,4.5,faible,4.5,4.5,4.5,NaN,24,NaN,NaN,NaN
9,350568242,73.0,BOULEVARD SAINT GERMAIN,75005,PARIS 5,1,LA SIRENE,SARL,9.0,PME,...,3.5,moyen,4.0,4.5,4.0,NaN,26,NaN,NaN,NaN
10,350908810,54.0,RUE DU MONT CENIS,75018,PARIS 18,1,SOURIRE DE SAIGON,SARL,8.0,PME,...,4.0,élevé,4.0,4.0,3.5,4.0,303,NaN,NaN,NaN
11,352677488,33.0,RUE ROUSSELET,75007,PARIS 7,1,LE SOLEIL DES ILES,Société créée de fait entre personnes physiques,0.0,PME,...,4.0,moyen,4.5,4.5,4.0,3.5,143,29.0,50.0,39.5
12,381324714,46.0,RUE DE LA CHAPELLE,75018,PARIS 18,1,HOUGGA-BENADDAD,Société créée de fait entre personnes physiques,0.0,PME,...,4.5,moyen,4.5,4.5,4.5,NaN,99,5.0,17.0,11.0


Dernier petit ajustement : pour le 16ème arrondissement, le code postal est souvent donné (à tort) comme "75116" alors qu'il devrait être égal à "75016". Conséquence : pour beaucoup de restaurants du 16ème arrondissement, on n'a pas toutes les données de l'arrondissement correspondant (car le matching Code INSEE, Code Postal ne s'est pas fait correctement, le code postal ayant été mal renseigné). On remedie facilement à cela, puisqu'on connaît toutes les données sur le 16ème arrondissement 

In [123]:
# Ex pour le resto ligne 24: problème car code postal 75116

parisiens.iloc[24]

legalImmat                                                           422292128
streetNumber                                                               141
streetName                                                  AVENUE DE MALAKOFF
Code Postal                                                              75116
city                                                                  PARIS 16
isHeadquarters                                                               1
legalName                                          ACG VILLA CORSE RIVE DROITE
legalStructure                                                            SARL
employeesNb                                                                  0
categorySize                                                               PME
foundingYear                                                              1999
sourceLastUpdate                                           2015-06-25T00:00:00
name                                               A

In [124]:
# Ex pour le resto ligne 86: pas de problème car bon code postal (75016)

parisiens.iloc[86]

legalImmat                                                           512447632
streetNumber                                                               135
streetName                                                     RUE DU RANELAGH
Code Postal                                                              75016
city                                                                  PARIS 16
isHeadquarters                                                               1
legalName                                                             LA PEPPA
legalStructure                                                            SARL
employeesNb                                                                  6
categorySize                                                               PME
foundingYear                                                              2009
sourceLastUpdate                                           2009-09-08T00:00:00
name                                                

En fait, ce ne sont que pour les mauvais codes postaux en 75116, que les informations sur les arrondissements ne sont pas renseignées( variables RT20,HT20,PTOT... ): 

In [125]:
parisiens[parisiens["RT20"].isnull()==True]

,legalImmat,streetNumber,streetName,Code Postal,city,isHeadquarters,legalName,legalStructure,employeesNb,categorySize,...,Note Globale,Catégorie de prix,Note de cuisine,Note de service,Note qualité-prix,Note ambiance,Nombre avis,Prix inf,Prix sup,Prix moyen
30,422292128,141.0,AVENUE DE MALAKOFF,75116,PARIS 16,1,ACG VILLA CORSE RIVE DROITE,SARL,0.0,PME,...,4.0,élevé,4.0,4.0,3.5,4.0,591,30.0,60.0,45.0
42,439077462,54.0,AVENUE KLEBER,75116,PARIS 16,1,WHATEVER,SARL,7.0,PME,...,4.5,faible,4.5,4.5,4.0,4.0,141,NaN,NaN,NaN
126,572180149,135.0,AVENUE DE MALAKOFF,75116,PARIS 16,1,LE BERLIOZ,SA,NaN,PME,...,4.0,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
145,793904681,50.0,RUE DECAMPS,75116,PARIS 16,1,DECAMPS,SAS,1.0,PME,...,4.5,moyen,4.5,4.5,4.0,4.5,457,13.0,18.0,15.5
152,801219403,54.0,RUE DE LONGCHAMP,75116,PARIS 16,0,DINAEL,SAS,0.0,PME,...,4.5,élevé,4.5,5.0,4.5,5.0,24,NaN,NaN,NaN
155,808238174,11.0,RUE DU BOUQUET DE LONGCHAMP,75116,PARIS 16,0,POP EXPEDITION,SAS,10.0,PME,...,4.0,moyen,4.0,4.0,4.0,NaN,102,14.0,35.0,24.5
169,820309912,26.0,RUE DE LA FAISANDERIE,75116,PARIS 16,1,FDC EXPANSION,SASU,NaN,NaN,...,4.0,élevé,4.0,4.0,3.0,4.0,877,50.0,150.0,100.0
174,823257225,25.0,RUE COPERNIC,75116,PARIS 16,1,BRASSERIE SICILIENNE,SAS,NaN,NaN,...,4.5,moyen,4.0,4.0,4.0,NaN,125,9.0,35.0,22.0
179,325414043,26.0,AVENUE PIERRE 1ER SERBIE,75116,PARIS 16,1,LA CHAUMIERE DE CHINE,SARL,3.0,PME,...,3.5,moyen,3.5,3.5,3.5,3.5,446,NaN,NaN,NaN
256,501604284,5.0,RUE DE LA MANUTENTION,75116,PARIS 16,1,L OASIS,SARL,6.0,PME,...,4.0,faible,4.5,4.0,5.0,NaN,15,NaN,NaN,NaN


In [126]:
 # On récupère les données du 16ème arrondissement :
    
Code_postal=parisiens.iloc[86].loc["Code Postal"]              # 75016 et non pas 75116
Code_INSEE=parisiens.iloc[86].loc["Code INSEE"]
Département=parisiens.iloc[86].loc["Département"]
Région=parisiens.iloc[86].loc["Région"]
Altitude_Moyenne=parisiens.iloc[86].loc["Altitude Moyenne"]                                                            
Superficie=parisiens.iloc[86].loc["Superficie"]                                                                
PTOT=parisiens.iloc[86].loc["PTOT"]                                                                   
Niveau_de_vie_Commune=parisiens.iloc[86].loc["Niveau de vie Commune"]                                                    
Niveau_de_vie_Département=parisiens.iloc[86].loc["Niveau de vie Département"]                                             
HT20=parisiens.iloc[86].loc["HT20"]                                                                        
HT120=parisiens.iloc[86].loc["HT120"]                                                                        
HT220=parisiens.iloc[86].loc["HT220"]                                                                        
HT320=parisiens.iloc[86].loc["HT320"]                                                                      
HT420=parisiens.iloc[86].loc["HT420"]          
HT520=parisiens.iloc[86].loc["HT520"]                                                                     
CPG20=parisiens.iloc[86].loc["CPG20"]                                                                       
RT20=parisiens.iloc[86].loc["RT20"]

In [127]:
# On remplace toutes les valeurs NaN par des -1 

parisiens["Code INSEE"].fillna(-1, inplace = True)
parisiens["Département"].fillna(-1, inplace = True)
parisiens["Région"].fillna(-1, inplace = True)
parisiens["Altitude Moyenne"].fillna(-1, inplace = True)
parisiens["Superficie"].fillna(-1, inplace = True)
parisiens["PTOT"].fillna(-1, inplace = True)
parisiens["Niveau de vie Commune"].fillna(-1, inplace = True)
parisiens["Niveau de vie Département"].fillna(-1, inplace = True)
parisiens["HT20"].fillna(-1, inplace = True)
parisiens["HT120"].fillna(-1, inplace = True)
parisiens["HT220"].fillna(-1, inplace = True)
parisiens["HT320"].fillna(-1, inplace = True)
parisiens["HT420"].fillna(-1, inplace = True)
parisiens["HT520"].fillna(-1, inplace = True)
parisiens["CPG20"].fillna(-1, inplace = True)
parisiens["RT20"].fillna(-1, inplace = True)

In [128]:
parisiens["Code Postal"]=parisiens["Code Postal"].apply(lambda s: Code_postal if s==75116 else s)
parisiens["Code INSEE"]=parisiens["Code INSEE"].apply(lambda s: Code_INSEE if s==-1 else s)
parisiens["Département"]=parisiens["Département"].apply(lambda s: Département if s==-1 else s)
parisiens["Région"]=parisiens["Région"].apply(lambda s: Région if s==-1 else s)
parisiens["Altitude Moyenne"]=parisiens["Altitude Moyenne"].apply(lambda s: Altitude_Moyenne if s==-1 else s)
parisiens["Superficie"]=parisiens["Superficie"].apply(lambda s: Superficie if s==-1 else s)
parisiens["PTOT"]=parisiens["PTOT"].apply(lambda s: PTOT if s==-1 else s)
parisiens["Niveau de vie Commune"]=parisiens["Niveau de vie Commune"].apply(lambda s: Niveau_de_vie_Commune if s==-1 else s)
parisiens["Niveau de vie Département"]=parisiens["Niveau de vie Département"].apply(lambda s: Niveau_de_vie_Département if s==-1 else s)
parisiens["HT20"]=parisiens["HT20"].apply(lambda s: HT20 if s==-1 else s)
parisiens["HT120"]=parisiens["HT120"].apply(lambda s: HT120 if s==-1 else s)
parisiens["HT220"]=parisiens["HT220"].apply(lambda s: HT220 if s==-1 else s)
parisiens["HT320"]=parisiens["HT320"].apply(lambda s: HT320 if s==-1 else s)
parisiens["HT420"]=parisiens["HT420"].apply(lambda s: HT420 if s==-1 else s)
parisiens["HT520"]=parisiens["HT520"].apply(lambda s: HT520 if s==-1 else s)
parisiens["CPG20"]=parisiens["CPG20"].apply(lambda s: CPG20 if s==-1 else s)
parisiens["RT20"]=parisiens["RT20"].apply(lambda s: RT20 if s==-1 else s)

In [129]:
parisiens.iloc[24]

legalImmat                                                           422292128
streetNumber                                                               141
streetName                                                  AVENUE DE MALAKOFF
Code Postal                                                              75016
city                                                                  PARIS 16
isHeadquarters                                                               1
legalName                                          ACG VILLA CORSE RIVE DROITE
legalStructure                                                            SARL
employeesNb                                                                  0
categorySize                                                               PME
foundingYear                                                              1999
sourceLastUpdate                                           2015-06-25T00:00:00
name                                               A

C'est bon cette fois on ales données du 16ème arronissement qui apparaissent 

#### On a enfin notre table finale !!

In [130]:
# On enregistre la table dans un fichier csv

parisiens.to_csv('C:/Users/GUILLOT Robin/Documents/Robin Ensae/Matières/Python/tablefinalisee.csv', sep = ';',index=False)

In [131]:
parisiens

,legalImmat,streetNumber,streetName,Code Postal,city,isHeadquarters,legalName,legalStructure,employeesNb,categorySize,...,Note Globale,Catégorie de prix,Note de cuisine,Note de service,Note qualité-prix,Note ambiance,Nombre avis,Prix inf,Prix sup,Prix moyen
0,304305519,5.0,AVENUE DU MAINE,75015,PARIS 15,1,LES CEDRES DU LIBAN,SARL,6.0,PME,...,4.5,moyen,4.5,4.5,4.0,3.5,850,8.0,21.0,14.5
1,306332362,25.0,RUE DE LA HUCHETTE,75005,PARIS 5,1,LE SOUVLAKI ATHENIEN,SARL,0.0,PME,...,4.0,faible,4.0,4.0,4.0,4.0,59,4.0,15.0,9.5
2,307711895,15.0,PLACE DU TERTRE,75018,PARIS 18,1,LA CREMAILLERE DE MONTMARTRE,SAS,20.0,PME,...,3.5,moyen,3.5,3.5,3.5,4.0,1036,17.0,37.0,27.0
3,318906591,11.0,RUE DE L ARRIVEE,75015,PARIS 15,0,LA BRIOCHE DOREE,SAS,1900.0,ETI,...,4.0,faible,4.0,3.5,4.0,3.5,421,NaN,NaN,NaN
4,319947537,14.0,RUE MABILLON,75006,PARIS 6,1,BRASSERIE DE LA GRILLE SAINT-GERMAIN,SAS,18.0,PME,...,3.0,moyen,3.0,3.5,3.0,3.5,336,NaN,NaN,NaN
6,331831826,87.0,RUE DE TURBIGO,75003,PARIS 3,1,SOC LE TEMPLE,SARL,5.0,PME,...,4.5,faible,4.5,4.5,4.5,NaN,24,NaN,NaN,NaN
9,350568242,73.0,BOULEVARD SAINT GERMAIN,75005,PARIS 5,1,LA SIRENE,SARL,9.0,PME,...,3.5,moyen,4.0,4.5,4.0,NaN,26,NaN,NaN,NaN
10,350908810,54.0,RUE DU MONT CENIS,75018,PARIS 18,1,SOURIRE DE SAIGON,SARL,8.0,PME,...,4.0,élevé,4.0,4.0,3.5,4.0,303,NaN,NaN,NaN
11,352677488,33.0,RUE ROUSSELET,75007,PARIS 7,1,LE SOLEIL DES ILES,Société créée de fait entre personnes physiques,0.0,PME,...,4.0,moyen,4.5,4.5,4.0,3.5,143,29.0,50.0,39.5
12,381324714,46.0,RUE DE LA CHAPELLE,75018,PARIS 18,1,HOUGGA-BENADDAD,Société créée de fait entre personnes physiques,0.0,PME,...,4.5,moyen,4.5,4.5,4.5,NaN,99,5.0,17.0,11.0


NB : on a assayé également de scrapper les prix moyens sur le site des pages jaunes. Mais l'algo, qui tournait bien, a été arrêté au bout de 500 itérations environ, et contrairement à Trip Advisor, nous sommes bloqués de façon permanente. 

Nous avons essayé d'exploiter des librairies comme Fake-User-agent, qui permettent de simuler avec un degré plus ou moins élevé ce que notre ordinateur envoie au site lorsqu'il s'agit d'un utilisateur réel. Nous avons un peu expériementé cette solution, en essayant par exemple de jouer sur les headers de la requête, avec un User-Agent répandu et pouvant appartenir à un vrai utilisateur. Ces tentatives n'ont pas marché sur le site des pages jaunes... Nous avons également essayé d'imposer des temps d'attente à notre webdriver, pour que celui-ci n'apparaisse pas trop "pressé" et ne soit pas directement classé comme robot ; sans succès.
On peut aller plus loin, en se servant de proxys gratuits par exemple, en jouant sur des adresses IP tournantes... mais on tombe dans un tout autre domaine...

#### Récupération prix moyen sur pages jaunes

In [ ]:
edge_options = EdgeOptions()                                   # Edge pour changer de Chrome...
edge_options.use_chromium = True
edge_options.add_argument("private")
edge_options.add_argument("--start-maximized");
edge_options.add_argument("--ignore-certificate-errors");
edge_options.add_argument("--disable-popup-blocking");
edge_options.add_argument("--incognito");





browser = Edge(executable_path="C:/Users/GUILLOT Robin/Documents/Robin Ensae/Matières/Python//msedgedriver",options=edge_options)




browser.get('https://www.pagesjaunes.fr/activites')

cookie=browser.find_element_by_id("didomi-notice-agree-button").click()

sleep(5)

prix_moy=[]


for k in tqdm(range(0,449)):                      # on a été bloqués au bout du 449ème restaurant
    
    nom=parisiens.iloc[k].loc['legalName']
    code_post=parisiens.iloc[k].loc['postalCode']
    
    search_bar_une = browser.find_element_by_id("quoiqui")
    search_bar_une.send_keys(nom)
    
    search_bar_deux = browser.find_element_by_id("ou")
    search_bar_deux.send_keys(code_post + Keys.ENTER)
        
    try:
        sleep(1)
        donnee = browser.find_element_by_xpath("//div[@class = 'zone-cvi-cviv']/p[1]").text
        prix_moy.append(donnee)
        
    except NoSuchElementException:
        prix_moy.append('NaN')
        
    search_bar_une_nettoyee = browser.find_element_by_id("quoiqui").clear()
    search_bar_deux_nettoyee = browser.find_element_by_id("ou").clear()
        
            
browser.quit()

In [23]:
prix_moy

['Budget : 21-30 euros',
 'Budget : 16-20 euros',
 'Budget : 31-45 euros',
 'Prestations : nettoyage écologique, repassage, teinturerie, nettoyage de tapis, nettoyage rideaux, nettoyage du daim, nettoyage du cuir, retouches, couture, parking, nettoyage robe de mariée, accès handicapé',
 'Budget : Inférieur à 10 euros',
 'Budget : 31-45 euros',
 'Budget : 31-45 euros',
 'NaN',
 'NaN',
 'Produits : Smoking, Costume, Chemise, Pantalon, Blazer, Parkas, Accessoires, Jeans, Polo, Manteaux, tenue de soirée',
 'Budget : 21-30 euros',
 'NaN',
 'NaN',
 'NaN',
 'Prestations : Clinique, Traitement des pathologies du genou, Chirurgie du genou, Sur rendez-vous, clinique de médecine du sport, service de radiologie, service de rhumatologie',
 'Prestations : terrasse, restaurant repas de groupe',
 'NaN',
 "Prestations : Vente en ligne ou par téléphone de billets de spectacles, Présentation en ligne d'événements",
 'Type de cuisine : restaurant oriental',
 'Type de cuisine : cuisine française, restauran